In [1]:
from pymongo import MongoClient
from elasticsearch import Elasticsearch
from basic.claims import connect #, get_claims, get_users, get_kibana_gate, products_bankiros, nulls, get_medium, get_claims_raw
# from basic.external import cpa_parse, get_yandex_data
# from basic.dscore import cre_get, mfo_score
from basic.reports import cre_keyword_report, master_report, registrations_medium
from basic.vars import regions_mapper, claim_offers
from bson.objectid import ObjectId
import requests
import json
from datetime import datetime, date, timedelta
import pandas as pd
from time import sleep
from base64 import b64encode
import numpy as np
# from robobrowser import RoboBrowser
import re
# import csv, sqlite3
# import sqlalchemy
import matplotlib.pyplot as plt
import seaborn as sns
from calendar import monthrange
import pickle
import os.path
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from dateutil.relativedelta import relativedelta
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth',1000)
# from IPython.display import Filelink, FileLinks


client, es = connect()

/opt/conda/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Доспуп к Эластику 	 True
Доспуп к Монге 		 True


In [2]:
def get_registrations(userIds):
    userIds = list(set(userIds))
    dp = pd.DataFrame()
    for i in range(int(len(userIds)/1000 + 1)):
        payload = {"size": 5000,
                   "query":{"bool":{"must":
                  [{"match_all":{}},
                  {"bool":{"should": [{"match_phrase":{"params.userId.keyword": _id }} for 
                                      _id in userIds[i*1000:(i+1)*1000]],
                                      "minimum_should_match":1 }},
                                     {"match_phrase":{"eventType.keyword":{"query":"REGISTRATION"}}}]}}}
        d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=60)
        print(d['hits']['total'], end="|")
        pt = {"card": "Кредитная карта",
              "Кредитная карта":"Кредитная карта",
              "cash": "Кредит наличными",
              "Кредит наличными": "Кредит наличными",
              "loan": "Займ",
              "Займ": "Займ",
              "refinance": "Рефинансирование",
              "Рефинансирование": "Рефинансирование",}

        dp1 = pd.DataFrame(d['hits']['hits'])
        dp = pd.concat([dp, dp1])
    dp['userId'] = dp['_source'].apply(lambda x: x['params']['userId'])
    dp['transaction_id'] = dp['_source'].apply(lambda x: x['params']['transaction_id'] if 
                        'transaction_id' in x['params'].keys() else np.nan)
    dp['productType'] = dp['_source'].apply(lambda x: x['params']['productType'] if 
                        'productType' in x['params'].keys() else np.nan)
    dp['productType'] = dp['productType'].map(pt)
    dp['utm_source'] = dp['_source'].apply(lambda x: x['params']['utm_source'] if 
                        'utm_source' in x['params'].keys() else np.nan)
    dp['utm_medium'] = dp['_source'].apply(lambda x: x['params']['utm_medium'] if 
                        'utm_medium' in x['params'].keys() else np.nan)
    dp['reg_time'] = dp['_source'].apply(lambda x: x['timestamp'] if 
                        'timestamp' in x.keys() else np.nan)
    dp['reg_time'] = pd.to_datetime(dp['reg_time']).apply(lambda x: x.replace(tzinfo=None))
#     print(dp['_source'].tolist())
    dp = dp[['userId', 'transaction_id', 'productType', 'utm_source', 'utm_medium', 'reg_time']]
    return dp

In [3]:
#!pip install robobrowser

In [4]:
# !pip install pickledb pymongo elasticsearch==6.8 oauth2client gspread df2gspread \
# robobrowser beautifulsoup4 lxml yourls Werkzeug==0.16.1 paramiko xlsxwriter pyminizip \
# html5lib zodiac_sign python-pptx plotly df2gspread gspread

In [5]:
#!pip install gspread_dataframe

In [6]:
def gsheet_api_check(SCOPES):
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)    
            
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)  # сюда json файл если он не в той же папке
            creds = flow.run_local_server(port=0)        
            
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)    
    return creds

def pull_sheet_data(SCOPES,SPREADSHEET_ID,RANGE_NAME):
    creds = gsheet_api_check(SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=RANGE_NAME).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range=RANGE_NAME).execute()
        data = rows.get('values')
        print("COMPLETE: Data copied")
        return data
    
def to_float(x):
    try:
        return(float(x.replace("\xa0", '').replace(" ", '').strip()))
    except Exception as e:
        return np.nan

In [7]:
def cre_get(es, since):
    
    chunk = 10000
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    payload = {"size": chunk,
               "_source": ["params.parsed"],
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword": "CRE"}},
                         {"range":{"timestamp":{"gte":gte,"format":"epoch_millis"}}}
                        ]}},
"_source": ["params.parsed", "timestamp"]
              }
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=3000)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/chunk)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=3000)
        hits = hits + d['hits']['hits']
        print(doc_count-chunk*i, end ="; ", flush=True)
        sleep(0.5)
#         break
    print(len(hits), 'raz')
    cre = []
    for h in hits:
        if len(h['_source']['params']['parsed'])>10:
            h2 = json.loads(h['_source']['params']['parsed'].replace("'", '"'))
            h2.update({"timestamp": pd.to_datetime(h['_source']['timestamp'])})
            cre.append(h2)
    df = pd.DataFrame(cre).sort_values("timestamp", 
                                       ascending=False).drop_duplicates('userId', keep='first').reset_index(drop=True)
    df.index = df['userId']
      
            
    return df['score'].to_dict()

In [8]:
def mm_requestId(x):
    if type(x)==dict and 'NOTE' in x.keys():
        if x['NOTE'].find('HTTP_RESPONSE_BODY') >0:
            htr = json.loads(x['NOTE'][x['NOTE'].find('HTTP_RESPONSE_BODY')+20:])
            htr = htr['requestId'] if 'requestId' in htr.keys() else np.nan
#             print(htr)
        else: 
            htr = np.nan
    else: 
        htr = np.nan
    return htr

In [9]:
def get_exact_claims(since, till, client):
    req = {"createdAt": {"$gte": datetime.combine(since, datetime.min.time()) - timedelta(hours=3),
                          "$lt": datetime.combine(till, datetime.min.time())- timedelta(hours=3)},
#            "product._id": {"$in": [ObjectId("5e9846be79dd27000141f476"),
# ObjectId("5ef4473acf22a90001010445")]},
           #"product._id": {"$in": [ObjectId(product) for product in products]}}
            "status": {"$nin": [ "ERROR", "NEW1"]}} 
    results_count = client['claim-service']['claims'].count_documents(req)
    # print(req)
    print(results_count)
    all_ans = []
    batch_size = 10000
    for i in range(results_count//batch_size+1):
        ans = list(client['claim-service']['claims'].find(req, {"product": 1, "partner":1,
                                                                "borrower": 1, "statusFields":1,
                                                                "status": 1, "adapterName": 1,
                                                                "createdAt": 1, 'type':1,
                                                                "updatedAt":1, "clickId": 1,
                                                                "whence":1}).limit(batch_size).skip(i*batch_size))
        all_ans = all_ans + ans
    #ans = trans(all_ans, client)
    df = pd.DataFrame(all_ans)
    df['id'] = df["_id"].apply(lambda x: str(x))
    df['dt'] = df["createdAt"].apply(lambda x: (x+timedelta(hours=3)).date())
    df['productId'] = df['product'].apply(lambda x: str(x['_id']))
    df['product_type'] = df['product'].apply(lambda x: x['productType'] if 'productType' in x.keys() else np.nan)
    df['product_name'] = df['product'].apply(lambda x: x['name'])
    df['partner'] = df['partner'].apply(lambda x: x['name'])
    df['userId'] = df['borrower'].apply(lambda x: x['userId'])
    df['phone'] = df['borrower'].apply(lambda x: x['phone'] if 'phone' in x.keys() else np.nan)

    return df


def get_kibana_claims(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 10000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"CLAIM"}},
#                          {"match_phrase":{"params.sourceGlobal.keyword":{"query": sourceGlobal }}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/10000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-10000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', #'payout', 'status',
                  'transaction_id', 'clickId', 'sourceGlobal', 'utm_content','orderId','transactionId']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['date'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

In [10]:
def get_kibana_postback(es, since):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(date.today()+timedelta(days=1), datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 10000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"POSTBACK"}},
#                          {"match_phrase":{"params.whence.keyword":{"query": whence }}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/10000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-10000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', 'payout', 'status',
                'transaction_id', 'externalId','orderId','transactionId']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['date'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

In [11]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000
# filters = "ym:s:UTMSource=='smsTopBP'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           #'utm_source':result[i]["dimensions"][1]["name"],
#             'visit-id':result[i]["dimensions"][3]["name"],
            #'area':result[i]["dimensions"][4]["name"],
#             'users':result[i]["metrics"][0],
             'visits':result[i]["metrics"][0],
            #'utm_term':result[i]["dimensions"][2]["name"],
#'avgVisitDurationSeconds':result[i]["dimensions"][3]["name"],
           #'yandexClientId':result[i]["dimensions"][5]["name"]
            #'startURL':result[i]["dimensions"][7]
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.strftime('%Y-%m')
df = df.groupby('month').agg({'visits':'sum'})
df = df.T
df = df.sort_index(axis='columns', level='month',ascending=False)
df.to_excel('Ежемесячный отчет/m_trafic_new_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1AVRkJrGTznlhpezr_2YxxtoyEjunlI_dP97uSCM-wx0')
worksheet = sh.get_worksheet(0) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Ежемесячный отчет/m_trafic_new_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)
df

INFO:root:Наличие семплирования: False


,Unnamed: 0,2021-12,2021-11,2021-10,2021-09,2021-08,2021-07,2021-06,2021-05,2021-04,2021-03,2021-02,2021-01
0,visits,37740,72183,65616,65295,43900,37488,53741,52633,89953,95822,89453,94813


In [12]:
#!pip install openpyxl

In [13]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:UTMSource",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000
# filters = "ym:s:UTMSource=='smsTopBP'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'utm_source':result[i]["dimensions"][1]["name"],
            #'visit-id':result[i]["dimensions"][2]["name"],
            #'utm_medium':result[i]["dimensions"][2]["name"],
       # 'utm_campaign':result[i]["dimensions"][3]["name"],
        #'utm_content':result[i]["dimensions"][4]["name"],
        #'utm_term':result[i]["dimensions"][5]["name"],
#             'users':result[i]["metrics"][0],
             'visits':result[i]["metrics"][0],
            #'utm_term':result[i]["dimensions"][2]["name"],
#'avgVisitDurationSeconds':result[i]["dimensions"][3]["name"],
           #'yandexClientId':result[i]["dimensions"][5]["name"]
            #'startURL':result[i]["dimensions"][7]
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.strftime('%Y-%m')

# df.to_excel('trafic_new_Alexander.xlsx')
# import pandas as pd
# import gspread
# from gspread_dataframe import set_with_dataframe

# # Доступ к GS
# gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
# sh = gc.open_by_key('1H70A5idclmnft2Pj3jzxjufAsHRofIgaOi-9uC_OMKU')
# worksheet = sh.get_worksheet(0) #-> 0 - лист один, 1 - второй и тд.

# # Выбор датафрейма
# df = pd.read_excel('trafic_new_Alexander.xlsx'
#                   )
# set_with_dataframe(worksheet, df)
# df

INFO:root:Наличие семплирования: False


In [14]:
df

,date,utm_source,visits,month
0,2021-01-04,None,2826.0,2021-01
1,2021-01-04,admitad,82.0,2021-01
2,2021-01-04,advcake,159.0,2021-01
3,2021-01-04,ais_google,141.0,2021-01
4,2021-01-04,ais_yandex,271.0,2021-01
...,...,...,...,...
7799,2021-12-20,sms_promo,1.0,2021-12
7800,2021-12-20,sms_trigger,12.0,2021-12
7801,2021-12-20,t2_kk,1.0,2021-12
7802,2021-12-20,tele2,35.0,2021-12


In [15]:
df_for_none = df

In [16]:
df_none=df_for_none[df_for_none['utm_source'].isnull()].groupby('month',as_index=False).agg({'visits':'sum'})
df_none

,month,visits
0,2021-01,80607.0
1,2021-02,77934.0
2,2021-03,77684.0
3,2021-04,71406.0
4,2021-05,29774.0
5,2021-06,25203.0
6,2021-07,22457.0
7,2021-08,29616.0
8,2021-09,37001.0
9,2021-10,46042.0


In [17]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:searchEngine",
    date1 ="2021-01-04",
    date2 = "yesterday",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000
    #filters = "ym:s:lastTrafficSource=='organic'"
# filters = "ym:s:UTMSource=='smsTopBP'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'utm_source':result[i]["dimensions"][1]["name"],
#             'visit-id':result[i]["dimensions"][3]["name"],
            #'area':result[i]["dimensions"][1]["name"],
#             'users':result[i]["metrics"][0],
             'visits':result[i]["metrics"][0],
            #'gender':result[i]["dimensions"][2]["name"],
#'avgVisitDurationSeconds':result[i]["dimensions"][3]["name"],
           #'yandexClientId':result[i]["dimensions"][5]["name"]
            #'startURL':result[i]["dimensions"][7]
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df_seo = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df_seo['date'] = pd.to_datetime(df_seo['date'])
df_seo['month'] = df_seo['date'].dt.strftime('%Y-%m')
df_seo = df_seo.groupby(['month','utm_source'],as_index=False).agg({'visits':'sum'})
df_seo = df_seo.groupby(['month'],as_index=False).agg({'visits':'sum'})

df_seo['Отдел']='SEO'
df_seo

INFO:root:Наличие семплирования: False


,month,visits,Отдел
0,2021-01,8867.0,SEO
1,2021-02,7478.0,SEO
2,2021-03,9378.0,SEO
3,2021-04,9736.0,SEO
4,2021-05,9138.0,SEO
5,2021-06,10093.0,SEO
6,2021-07,7847.0,SEO
7,2021-08,12147.0,SEO
8,2021-09,14572.0,SEO
9,2021-10,19968.0,SEO


In [18]:
df_none1 = df_none.merge(df_seo,how='left',on='month')
df_none1['visits'] = df_none1['visits_x']- df_none1['visits_y']
df_none2= df_none1[['month','visits']]
df_none2['utm_source']='Прочее'
df_none2

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,month,visits,utm_source
0,2021-01,71740.0,Прочее
1,2021-02,70456.0,Прочее
2,2021-03,68306.0,Прочее
3,2021-04,61670.0,Прочее
4,2021-05,20636.0,Прочее
5,2021-06,15110.0,Прочее
6,2021-07,14610.0,Прочее
7,2021-08,17469.0,Прочее
8,2021-09,22429.0,Прочее
9,2021-10,26074.0,Прочее


In [19]:
df=df[~df['utm_source'].isnull()]
df = df.groupby(['month','utm_source'],as_index=False).agg({'visits':'sum'})
df= pd.concat([df,df_none2])
df

,month,utm_source,visits
0,2021-01,admitad,4334.0
1,2021-01,advcake,3067.0
2,2021-01,advertise,1.0
3,2021-01,ais_google,1055.0
4,2021-01,ais_yandex,2768.0
5,2021-01,bank-zenit,6.0
6,2021-01,bankiros,2.0
7,2021-01,callcenter,7.0
8,2021-01,content,25.0
9,2021-01,content-text,11.0


In [20]:
#df['utm_source']=df['utm_source'].fillna('Не определено')

In [21]:
#df

In [22]:
df['utm_source'] = df['utm_source'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Vbr.ru ссылки' if 
                                  x in ['vbrru', 'upside_vbr','vbrr','vbr'] else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Email' if 
                                 x[:5]=='email' or x[:7]=='TRIGGER' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'SMS рекламные' if 
                                   x[:3]=='sms' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'SMS рекламные' if 
                                   x in ['tele2TommorowWON',
                                         'tele2Tommorow',
                                         'tele2FourDays',
                                         'tele2TOP'] or x[:5]=='t2sms' or x =='lostBasket' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Admitad' if 
                                   x[:7]=='admitad' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Adv.Cake' if 
                                   x[:7]=='advcake' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Leadgid' if 
                                   x[:7]=='leadgid' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Rubid' if 
                                   x[:5]=='rubid' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Cpahub' if 
                                   x[:6]=='cpahub' or x[:6]=='Cpahub' or x== 'get4click'
                                          or x =='checkscan' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Перфоманс через Артикс' if 
                                   x[:3]=='ais' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Tele2 // Микрозаймы' if 
                                   x[:4]=='tele' or x[:2]=='t2' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Виджеты' if 
                                   x[:10]=='substitute' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Браузерные пуши (web)' if 
                                   x[:4]=='push' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Exbico' if 
                                   x[:6]=='exbico' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Ссылочная масса' if 
                                   x[:8]=='showcase' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Статьи на сайте' if 
                                   x[:12]=='content' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'МТС' if 
                                   x[:2]=='mt' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Brobank.ru' if 
                                   x[:7]=='brobank' else x)
df['utm_source'] = df['utm_source'].apply(lambda x: 'Тестирование гипотез' if 
                                  x[:6]=='yandex' or x[:6]=='google' else x)
# # df['utm_source'] = dr.apply(lambda row: 'Органический трафик' if row['utm_source']=='Ссылочная масса' 
# #                         else row['utm_source'], axis=1)
# df['utm_source'] = dr.apply(lambda row: 'New (2Gis)' if row['utm_source']=='2gis' 
#                         else row['utm_source'], axis=1)
# df['utm_source'] = dr.apply(lambda row: 'Мобильное приложение 1' if row['utm_source']=='Мобильное приложение' 
#                         else row['utm_source'], axis=1)

In [23]:
df

,month,utm_source,visits
0,2021-01,Admitad,4334.0
1,2021-01,Adv.Cake,3067.0
2,2021-01,advertise,1.0
3,2021-01,Перфоманс через Артикс,1055.0
4,2021-01,Перфоманс через Артикс,2768.0
5,2021-01,bank-zenit,6.0
6,2021-01,Bankiros.ru ссылки,2.0
7,2021-01,callcenter,7.0
8,2021-01,Статьи на сайте,25.0
9,2021-01,content-text,11.0


In [24]:
df['Отдел'] = df['utm_source']

In [25]:
df['Отдел'] = df['Отдел'].apply(lambda x: 'Direct' if 
                                   x in ['Email','SMS рекламные','Браузерные пуши (web)'] else x)
df['Отдел'] = df['Отдел'].apply(lambda x: 'SEO' if 
                                   x in ['Наполнение лендингов по городам',
                                         'Органический трафик','продвижение МП'] else x)
df['Отдел'] = df['Отдел'].apply(lambda x: 'SEO' if 
                                   x in ['Наполнение лендингов по городам',
                                         'Органический трафик','продвижение МП',
                                         'накрутка поведенческих факторов',
                                         'Мобильное приложение 1','Мобильное приложение',
                                         'Новая карточка',
                                         'Новые витрины'] else x)
df['Отдел'] = df['Отдел'].apply(lambda x: 'CPA' if 
                                   x in ['Adv.Cake','Admitad','Leadgid','Cpahub','Rubid','Город Тройка',
                                         'Мобильное приложение',
                                         'Металлинвест \ потребы','CPA пиар','Новые продукты \\ дебетовые',
                                         'CallTat','Металлинвест \\ РКО','Промокоды','Чек Скан'] else x)
df['Отдел'] = df['Отдел'].apply(lambda x: 'Своя CPA' if 
                                   x in ['Bankiros.ru ссылки','Vbr.ru WL','Vbr.ru ссылки','Brobank.ru',
                                         'Металлинвест','Bankiros.ru WL','Новые партнеры WL'] else x)
df['Отдел'] = df['Отдел'].apply(lambda x: 'Контент' if 
                                   x in ['Яндекс.Дзен','Статьи на сайте',
                                         'Рекламные статьи (платные публикации)'] else x)
df['Отдел'] = df['Отдел'].apply(lambda x: 'Крупные партнеры' if 
                                   x in ['Tele2 // Микрозаймы','Теле2 \\ Кредитные карты',
                                         'Qiwi \\ Микрозайм','New (2Gis)','Мегафон','Крупный партнер (1)',
                                         'Крупный партнер (2)','Крупный партнер (3)'] else x)
df['Отдел'] = df['Отдел'].apply(lambda x: 'КЦ' if 
                                   x in ['Exbico','Внешние базы','Обзвон роботом','Допродажи КЦ'] else x)
df['Отдел'] = df['Отдел'].apply(lambda x: 'Перфоманс' if 
                                   x in ['Перфоманс','Перфоманс через Артикс','МТС \\  баннер + СМС',
                                         'Пуш партнерка','Тестирование гипотез'] else x)
df['Отдел'] = df['Отдел'].apply(lambda x: 'Продукт' if 
                                   x in ['Быстрая регистрация','Подписка','Конкурс',
                                         'Энвибокс','Новые продукты \\ Подборщик дебетовый карт',
                                         'Новые продукты \\ РКО'] else x)
df['Отдел'] = df['Отдел'].apply(lambda x: 'Продукт' if 
                                   x in ['Быстрая регистрация','Подписка','Конкурс',
                                         'Энвибокс','Новые продукты \\ Подборщик дебетовый карт',
                                         'Новые продукты \\ РКО'] else x)
df['Отдел'] = df['Отдел'].apply(lambda x: 'Прочее' if 
                                   x in ['stream_direct','propeller','evadav', 'Не определено'] else x)


In [26]:
df

,month,utm_source,visits,Отдел
0,2021-01,Admitad,4334.0,CPA
1,2021-01,Adv.Cake,3067.0,CPA
2,2021-01,advertise,1.0,advertise
3,2021-01,Перфоманс через Артикс,1055.0,Перфоманс
4,2021-01,Перфоманс через Артикс,2768.0,Перфоманс
5,2021-01,bank-zenit,6.0,bank-zenit
6,2021-01,Bankiros.ru ссылки,2.0,Своя CPA
7,2021-01,callcenter,7.0,callcenter
8,2021-01,Статьи на сайте,25.0,Контент
9,2021-01,content-text,11.0,content-text


In [27]:
df = df[['month','visits','Отдел']]
df

,month,visits,Отдел
0,2021-01,4334.0,CPA
1,2021-01,3067.0,CPA
2,2021-01,1.0,advertise
3,2021-01,1055.0,Перфоманс
4,2021-01,2768.0,Перфоманс
5,2021-01,6.0,bank-zenit
6,2021-01,2.0,Своя CPA
7,2021-01,7.0,callcenter
8,2021-01,25.0,Контент
9,2021-01,11.0,content-text


In [28]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:searchEngine",
    date1 ="2021-01-04",
    date2 = "yesterday",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000
    #filters = "ym:s:lastTrafficSource=='organic'"
# filters = "ym:s:UTMSource=='smsTopBP'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'utm_source':result[i]["dimensions"][1]["name"],
#             'visit-id':result[i]["dimensions"][3]["name"],
            #'area':result[i]["dimensions"][1]["name"],
#             'users':result[i]["metrics"][0],
             'visits':result[i]["metrics"][0],
            #'gender':result[i]["dimensions"][2]["name"],
#'avgVisitDurationSeconds':result[i]["dimensions"][3]["name"],
           #'yandexClientId':result[i]["dimensions"][5]["name"]
            #'startURL':result[i]["dimensions"][7]
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df_seo = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df_seo['date'] = pd.to_datetime(df_seo['date'])
df_seo['month'] = df_seo['date'].dt.strftime('%Y-%m')
df_seo = df_seo.groupby(['month','utm_source'],as_index=False).agg({'visits':'sum'})
df_seo = df_seo.groupby(['month'],as_index=False).agg({'visits':'sum'})

df_seo['Отдел']='SEO'
df_seo

INFO:root:Наличие семплирования: False


,month,visits,Отдел
0,2021-01,8867.0,SEO
1,2021-02,7478.0,SEO
2,2021-03,9378.0,SEO
3,2021-04,9736.0,SEO
4,2021-05,9138.0,SEO
5,2021-06,10093.0,SEO
6,2021-07,7847.0,SEO
7,2021-08,12147.0,SEO
8,2021-09,14572.0,SEO
9,2021-10,19968.0,SEO


In [29]:
def get_kibana_screensplash(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 5000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"SPLASH_SCREEN"}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}},
"_source":["timestamp", "params.yandexClientID", "params.userId", "params.userId",
           "params.currentPage", "params.gaId","params.sourceGlobal"]}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.2)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/5000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-5000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr = dr[dr['params'].notnull()]
#     dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId','utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID', 'currentPage',
                 'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', 'payout', 'status',
                   'transaction_id', 'clickId', 'sourceGlobal'
                ]:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)
    del dr['params']

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['dt'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

In [30]:
since = date(2021,1,1)
till=date(2021,12,31)
screen_splash = get_kibana_screensplash(es, since,till)

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/events/event/_search?scroll=120m [status:200 request:3.644s]


37772; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.979s]


32772; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.640s]


...

12772; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:4.601s]


7772; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.407s]


2772; 

In [31]:
screen_splash['dt']=pd.to_datetime(screen_splash['dt'])
screen_splash['month'] = screen_splash['dt'].dt.strftime('%Y-%m')
dg1 = screen_splash.groupby(['month'], as_index=False).agg({'dt':'count'})
dg1['Отдел']='Мобильное приложение 1'
dg1.columns = ['month','visits','Отдел']
dg1

,month,visits,Отдел
0,2021-01,546,Мобильное приложение 1
1,2021-02,797,Мобильное приложение 1
2,2021-03,2224,Мобильное приложение 1
3,2021-04,5779,Мобильное приложение 1
4,2021-05,3364,Мобильное приложение 1
5,2021-06,2871,Мобильное приложение 1
6,2021-07,2988,Мобильное приложение 1
7,2021-08,3855,Мобильное приложение 1
8,2021-09,4043,Мобильное приложение 1
9,2021-10,3858,Мобильное приложение 1


In [32]:
df_traf= pd.concat([df,df_seo])
df_traf = pd.concat([df_traf,dg1])
df_traf

,month,visits,Отдел
0,2021-01,4334.0,CPA
1,2021-01,3067.0,CPA
2,2021-01,1.0,advertise
3,2021-01,1055.0,Перфоманс
4,2021-01,2768.0,Перфоманс
5,2021-01,6.0,bank-zenit
6,2021-01,2.0,Своя CPA
7,2021-01,7.0,callcenter
8,2021-01,25.0,Контент
9,2021-01,11.0,content-text


In [33]:
df_fin = df_traf.pivot_table(index='Отдел',columns='month',values='visits',aggfunc='sum')

df_fin = df_fin.fillna(0)

df_fin = df_fin.sort_index(axis='columns', level='month',ascending=False)


df_fin['sum'] = df_fin.sum(axis=1)


In [34]:
df_fin = df_fin[df_fin['sum']>500]
df_fin  = df_fin.sort_values('Отдел',ascending=False)
df_fin.to_excel('Ежемесячный отчет/m_trafic_block_new_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1AVRkJrGTznlhpezr_2YxxtoyEjunlI_dP97uSCM-wx0')
worksheet = sh.get_worksheet(1) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Ежемесячный отчет/m_trafic_block_new_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)


In [35]:
df_fin

month,2021-12,2021-11,2021-10,2021-09,2021-08,2021-07,2021-06,2021-05,2021-04,2021-03,2021-02,2021-01,sum
Отдел,,,,,,,,,,,,,
Своя CPA,17.0,33.0,29.0,42.0,95.0,111.0,221.0,302.0,255.0,289.0,448.0,89.0,1931.0
Прочее,16820.0,23165.0,26092.0,22450.0,17977.0,14641.0,15375.0,24863.0,61670.0,68306.0,70456.0,71740.0,433555.0
Перфоманс,3246.0,3016.0,5772.0,6102.0,4913.0,4181.0,14963.0,5588.0,4862.0,916.0,1226.0,4285.0,59070.0
Мобильное приложение 1,2928.0,4441.0,3858.0,4043.0,3855.0,2988.0,2871.0,3364.0,5779.0,2224.0,797.0,546.0,37694.0
Крупные партнеры,2212.0,15138.0,7190.0,4975.0,5457.0,6791.0,7692.0,3204.0,4759.0,6590.0,1874.0,77.0,65959.0
test,13.0,569.0,0.0,0.0,3.0,6.0,15.0,0.0,0.0,0.0,0.0,0.0,606.0
content-text,47.0,38.0,36.0,58.0,49.0,69.0,45.0,52.0,57.0,57.0,19.0,11.0,538.0
cityads,587.0,408.0,355.0,97.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1449.0
SEO,12593.0,19992.0,19968.0,14572.0,12147.0,7847.0,10093.0,9138.0,9736.0,9378.0,7478.0,8867.0,141809.0


In [36]:
def get_kibana_reg(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 1000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"REGISTRATION"}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/1000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-1000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID','currentPage',
#                 'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', #'payout', 'status',
#                   'transaction_id', 'clickId', 
                'sourceGlobal']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['dt_reg'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

regs = get_kibana_reg(es, since,till)


INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/events/event/_search?scroll=120m [status:200 request:0.555s]


35683; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.118s]


34683; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.039s]


...

2683; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.222s]


1683; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.117s]


683; 

In [37]:
regs['dt_reg'] = pd.to_datetime(regs['dt_reg'])
regs['month']  =regs['dt_reg'].dt.strftime('%Y-%m')

In [38]:
regs['utm_source']=regs.apply(lambda row: row['utm_source'] if pd.notnull(row['utm_source']) else row['sourceGlobal'],axis=1)

In [39]:
regs['Отдел']= regs['utm_source']

In [40]:
regs['Отдел']=regs.apply(lambda row: row['Отдел'] if 
                      pd.notnull(row['Отдел']) else row['utm_medium'],axis=1)

In [41]:
regs['Отдел'] = regs.apply(lambda row: 'Мобильное приложение 1' if row['Отдел']=='Android_mobile' 
                        else row['Отдел'], axis=1)
regs['Отдел'] = regs.apply(lambda row: 'Мобильное приложение 1' if row['Отдел']=='Huawei_mobile' 
                        else row['Отдел'], axis=1)
regs['Отдел'] = regs.apply(lambda row: 'Мобильное приложение 1' if row['Отдел']=='Apple_mobile' 
                        else row['Отдел'], axis=1)
regs['Отдел'] = regs.apply(lambda row: 'SEO' if row['Отдел']=='organic' 
                        else row['Отдел'], axis=1)

In [42]:
regs['Отдел']=regs['Отдел'].fillna('Не определено')

In [43]:
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Vbr.ru ссылки' if 
                                  x in ['vbrru', 'upside_vbr','vbrr','vbr'] else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Email' if 
                                 x[:5]=='email' or x[:7]=='TRIGGER' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'SMS рекламные' if 
                                   x[:3]=='sms' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'SMS рекламные' if 
                                   x in ['tele2TommorowWON',
                                         'tele2Tommorow',
                                         'tele2FourDays',
                                         'tele2TOP'] or x[:5]=='t2sms' or x =='lostBasket' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Admitad' if 
                                   x[:7]=='admitad' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Adv.Cake' if 
                                   x[:7]=='advcake' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Leadgid' if 
                                   x[:7]=='leadgid' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Rubid' if 
                                   x[:5]=='rubid' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Cpahub' if 
                                   x[:6]=='cpahub' or x[:6]=='Cpahub' or x== 'get4click'
                                          or x =='checkscan' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Перфоманс через Артикс' if 
                                   x[:3]=='ais' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Tele2 // Микрозаймы' if 
                                   x[:4]=='tele' or x[:2]=='t2' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Виджеты' if 
                                   x[:10]=='substitute' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Браузерные пуши (web)' if 
                                   x[:4]=='push' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Exbico' if 
                                   x[:6]=='exbico' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Ссылочная масса' if 
                                   x[:8]=='showcase' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Статьи на сайте' if 
                                   x[:12]=='content' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'МТС' if 
                                   x[:2]=='mt' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Brobank.ru' if 
                                   x[:7]=='brobank' else x)
regs['Отдел'] = regs['Отдел'].apply(lambda x: 'Тестирование гипотез' if 
                                  x[:6]=='yandex' or x[:6]=='google' else x)
# # df['utm_source'] = dr.apply(lambda row: 'Органический трафик' if row['utm_source']=='Ссылочная масса' 
# #                      else row['utm_source'], axis=1)
# df['utm_source'] = dr.apply(lambda row: 'New (2Gis)' if row['utm_source']=='2gis' 
#                         else row['utm_source'], axis=1)
#df['utm_source'] = dr.apply(lambda row: 'Мобильное приложение 1' if row['sourceGlobal'] in ['Huawei_mobile','Apple_mobile','Android_mobile'] else row['Отдел'], axis=1)

In [44]:
df_reg = regs

In [45]:
df_reg['Отдел'].unique()

array(['Adv.Cake', 'Перфоманс через Артикс', 'SEO', 'Не определено',
       'Brobank.ru', 'SMS рекламные', 'Admitad', 'Тестирование гипотез',
       'Leadgid', 'kupikupon', 'content-text', 'Exbico',
       'Браузерные пуши (web)', 'Email', 'Tele2 // Микрозаймы',
       'Bankiros.ru ссылки', 'credit_cards', 'combomoney',
       'mickrozaim.ru', 'Мобильное приложение 1', 'text', 'test_partner',
       'rubanks', 'Finmarket_loan', 'Rubid', 'ITstrahovka', 'sberometer',
       'Cpahub', 'evadav', 'Finmarket_cash', '', 'stream_direct',
       'cityads', 'Статьи на сайте', 'Finmarket_card', 'guruleads',
       'landing_odobrim', 'megafon_channels', 'retail', 'landing',
       'cameo', 'Dengio_card'], dtype=object)

In [46]:
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'Direct' if 
                                   x in ['Email','SMS рекламные','Браузерные пуши (web)'] else x)
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'SEO' if 
                                   x in ['Наполнение лендингов по городам',
                                         'Органический трафик','продвижение МП'] else x)
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'SEO' if 
                                   x in ['Наполнение лендингов по городам',
                                         'Органический трафик','продвижение МП',
                                         'накрутка поведенческих факторов',
                                         'Новая карточка',
                                         'Новые витрины','content-text'] else x)
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'CPA' if 
                                   x in ['Adv.Cake','Admitad','Leadgid','Cpahub','Rubid','Город Тройка',
                                         'Мобильное приложение',
                                         'Металлинвест \ потребы','CPA пиар','Новые продукты \\ дебетовые',
                                         'CallTat','Металлинвест \\ РКО','Промокоды','Чек Скан'] else x)
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'Своя CPA' if 
                                   x in ['Bankiros.ru ссылки','Vbr.ru WL','Vbr.ru ссылки','Brobank.ru',
                                         'Металлинвест','Bankiros.ru WL','Новые партнеры WL'] else x)
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'Контент' if 
                                   x in ['Яндекс.Дзен','Статьи на сайте',
                                         'Рекламные статьи (платные публикации)'] else x)
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'Крупные партнеры' if 
                                   x in ['Tele2 // Микрозаймы','Теле2 \\ Кредитные карты',
                                         'Qiwi \\ Микрозайм','New (2Gis)','Мегафон','Крупный партнер (1)',
                                         'Крупный партнер (2)','Крупный партнер (3)','test_partner'] else x)
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'КЦ' if 
                                   x in ['Exbico','Внешние базы','Обзвон роботом','Допродажи КЦ'] else x)
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'Перфоманс' if 
                                   x in ['Перфоманс','Перфоманс через Артикс','МТС \\  баннер + СМС',
                                         'Пуш партнерка','Тестирование гипотез'] else x)
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'Продукт' if 
                                   x in ['Быстрая регистрация','Подписка','Конкурс',
                                         'Энвибокс','Новые продукты \\ Подборщик дебетовый карт',
                                         'Новые продукты \\ РКО'] else x)
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'Продукт' if 
                                   x in ['Быстрая регистрация','Подписка','Конкурс',
                                         'Энвибокс','Новые продукты \\ Подборщик дебетовый карт',
                                         'Новые продукты \\ РКО'] else x)
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'Мобильное приложение 1' if 
                                   x in ['Мобильное приложение 1'] else x)
df_reg['Отдел'] = df_reg['Отдел'].apply(lambda x: 'Прочее' if 
                                   x in ['stream_direct','propeller','evadav', 'Не определено'] else x)


In [47]:
df_fin1 = df_reg.pivot_table(index='Отдел',columns='month',values='userId',aggfunc='count')

df_fin1 = df_fin1.fillna(0)

df_fin1 = df_fin1.sort_index(axis='columns', level='month',ascending=False)


df_fin1['sum'] = df_fin1.sum(axis=1)

In [48]:
df_fin1 = df_fin1[df_fin1['sum']>100]
df_fin1 = df_fin1.sort_values('Отдел',ascending=False)

In [49]:
df_fin1.to_excel('Ежемесячный отчет/m_reg_new_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1AVRkJrGTznlhpezr_2YxxtoyEjunlI_dP97uSCM-wx0')
worksheet = sh.get_worksheet(2) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Ежемесячный отчет/m_reg_new_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)

In [50]:
df_fin1

month,2021-12,2021-11,2021-10,2021-09,2021-08,2021-07,2021-06,2021-05,2021-04,2021-03,2021-02,2021-01,sum
Отдел,,,,,,,,,,,,,
Своя CPA,65.0,124.0,96.0,109.0,171.0,215.0,326.0,612.0,794.0,692.0,532.0,524.0,4260.0
Прочее,201.0,277.0,244.0,232.0,281.0,601.0,775.0,1085.0,1046.0,1017.0,665.0,598.0,7022.0
Перфоманс,1.0,9.0,7.0,3.0,8.0,15.0,41.0,42.0,44.0,33.0,29.0,85.0,317.0
Мобильное приложение 1,826.0,1473.0,1083.0,1127.0,1167.0,665.0,606.0,773.0,1937.0,551.0,86.0,10.0,10304.0
Крупные партнеры,5.0,29.0,231.0,12.0,28.0,289.0,31.0,48.0,73.0,51.0,1.0,3.0,801.0
cityads,5.0,30.0,116.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,173.0
SEO,103.0,176.0,141.0,135.0,123.0,127.0,180.0,170.0,234.0,271.0,193.0,200.0,2053.0
Direct,13.0,12.0,15.0,13.0,10.0,6.0,5.0,24.0,33.0,56.0,19.0,40.0,246.0
CPA,506.0,1215.0,1507.0,1959.0,256.0,570.0,506.0,919.0,594.0,783.0,613.0,752.0,10180.0


In [51]:
df_traf = df_traf.groupby(['Отдел','month'],as_index=False).agg({'visits':'sum'})
df_traf = pd.concat([df_traf,df_seo])
df_traf

,Отдел,month,visits
0,213,2021-11,1.0
1,CPA,2021-01,7785.0
2,CPA,2021-02,6517.0
3,CPA,2021-03,6899.0
4,CPA,2021-04,5971.0
5,CPA,2021-05,7963.0
6,CPA,2021-06,4528.0
7,CPA,2021-07,3178.0
8,CPA,2021-08,2738.0
9,CPA,2021-09,16337.0


## Табличка трафик-рега

In [52]:
df_regs = df_reg.groupby(['Отдел','month'],as_index=False).agg({'userId':'count'})
df_regs = df_regs[df_regs['Отдел']!='']

In [53]:
cr_reg = df_regs.merge(df_traf,how='left',on=['Отдел','month'])

In [54]:
cr_reg['CR_Трафик-->Регистрация'] = ((cr_reg['userId']/cr_reg['visits'])*100).round(2)
cr_reg = cr_reg[cr_reg['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено','Контент','КЦ'])]

In [55]:
cr_reg_pivot = cr_reg.pivot_table(index='Отдел',
                                  columns='month'
                                  ,values='CR_Трафик-->Регистрация').sort_index(axis='columns',
                                                                                level='month',
                                                                                ascending=False)
cr_reg_pivot = cr_reg_pivot.fillna(0)
cr_reg_pivot = cr_reg_pivot.sort_values('Отдел',ascending=False)
cr_reg_pivot.to_excel('Ежемесячный отчет/m_reg_traf_cr_new_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1AVRkJrGTznlhpezr_2YxxtoyEjunlI_dP97uSCM-wx0')
worksheet = sh.get_worksheet(3) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Ежемесячный отчет/m_reg_traf_cr_new_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)

In [56]:
cr_reg_pivot

month,2021-12,2021-11,2021-10,2021-09,2021-08,2021-07,2021-06,2021-05,2021-04,2021-03,2021-02,2021-01
Отдел,,,,,,,,,,,,
Своя CPA,382.35,375.76,331.03,259.52,180.00,193.69,147.51,202.65,311.37,239.45,118.75,588.76
Перфоманс,0.03,0.30,0.12,0.05,0.16,0.36,0.27,0.75,0.90,3.60,2.37,1.98
Мобильное приложение 1,28.21,33.17,28.07,27.88,30.27,22.26,21.11,22.98,33.52,24.78,10.79,1.83
Крупные партнеры,0.23,0.19,3.21,0.24,0.51,4.26,0.40,1.50,1.53,0.77,0.05,3.90
Контент,0.00,0.00,0.00,0.00,4.76,0.00,0.00,0.00,0.00,0.00,0.00,0.00
КЦ,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.63,0.00,3.70,4.11,0.88
SEO,0.82,0.88,0.71,0.93,1.01,1.62,1.78,1.86,2.40,2.89,2.58,2.26
Direct,3.60,0.60,3.54,2.73,2.60,1.08,1.16,2.69,1.39,1.86,1.69,2.53
CPA,16.43,16.02,28.69,11.99,9.35,17.94,11.17,11.54,9.95,11.35,9.41,9.66


In [57]:
df = get_exact_claims(since, till, client)
df['requestId'] = df[df['partner']=='MoneyMan']['statusFields'].apply(mm_requestId)
dpbk = get_kibana_postback(es, since)
def float_apply(x):
    try:
        return float(x)
    except ValueError:
        return None
dpbk['payout'] = dpbk['payout'].apply(float_apply)
dpbk['payout'] = dpbk.apply(lambda x: x['payout']*1.2 if x['partnerName']=='admitad' else x['payout'], axis=1)
dpbk = dpbk[dpbk['status'].isin(['APPROVED', 'approved', 'ISSUED'])].sort_values('payout').drop_duplicates('transitionId', keep='last')
dr = pd.merge(left=df, right=dpbk, left_on='id', right_on='transitionId', how='left')
dc = get_kibana_claims(es, since,till)[['clickId', 'userId', 'utm_campaign', 'utm_medium', 'utm_content', 
                                        'utm_source', 'utm_term', 'gaId', 'yandexClientID', 'sourceGlobal']]
dr = pd.merge(left=dr, right=dc[dc['clickId'].notnull()].drop_duplicates('clickId'), on='clickId', how='left')

cre = cre_get(es, since)
dr['score'] = dr['userId_x'].map(cre).fillna(0).astype(int)

dr.to_pickle(f'reports/{datetime.strftime(since, "%B")}_claimReport.pickle.gz')
dc.to_pickle(f'reports/{datetime.strftime(since, "%B")}_claims.pickle.gz')

def AR_count(score, productType, status):
    koef = 1
    check = [2200, 1700, 1100, 5500]
    ARlist = [[0.00499500499500499, 0.00305343511450382, 0.0714285714285714, 0.0], 
              [0.00380818053596615, 0.00871508379888268, 0.0768345323741007, 0.0], 
              [0.00685378590078329, 0.0136418349605457, 0.0846340617412418, 0.0053475935828877], 
              [0.0111028867505552, 0.0255610972568579, 0.0980751604032997, 0.0116279069767442], 
              [0.00938967136150235, 0.0349985043374215, 0.126042632066728, 0.0], 
              [0.0109689213893967, 0.0407338308457712, 0.140204271123491, 0.0], 
              [0.0141884222474461, 0.0514050719671008, 0.113486842105263, 0.0188679245283019], 
              [0.0150753768844221, 0.0687361419068736, 0.0743243243243243, 0.0], 
              [0.00943396226415094, 0.0415430267062315, 0.0892857142857143, 0.0],
              [0.00943396226415094, 0.0415430267062315, 0.0892857142857143, 0.0]]
    
    if score is np.nan or status in ['NEW', 'ERROR']:
        return 0
    elif score > 10 and productType == 'CASH_LOAN':
        return round(ARlist[int(round(score, -1)/10)-1][0]*check[0]*koef, 2)
    elif score > 10 and productType == 'CREDIT_CARD':
        return round(ARlist[int(round(score, -1)/10)-1][1]*check[1]*koef, 2)
    elif score > 10 and productType == 'LOAN':
#         print(score, productType, status)
        return round(ARlist[int(round(score, -1)/10)-1][2]*check[2]*koef, 2)
    elif score > 10 and productType == 'REFINANCE':
        return round(ARlist[int(round(score, -1)/10)-1][3]*check[3]*koef, 2)
    else:
        return round(ARlist[int(round(score, -1)/10)-1][0]*check[0]*koef, 2)

dr['createdAt'] = pd.to_datetime(dr['createdAt'])
dr['payout'] = dr['payout'].astype(float)
dr['payout'] = dr.apply(lambda row: 3800 if row['partner']=='Альфа-Банк' 
                        else row['payout'], axis=1)
dr['payout'] = dr.apply(lambda row: 2550 if row['partner']=='Kredito24' 
                        else row['payout'], axis=1)
dr['productType'] = dr['product'].apply(lambda x: x['productType'])


dr['source'] = dr.apply(lambda row: 'Металинвест Cash' if row['productId_x']=='5e9846be79dd27000141f476' 
                        else row['source'], axis=1)

#dr = dr[dr['status_x']=='NEW']
dr['score'] = dr['score'].apply(lambda x: x+75 if x==0 else x)
dr['whence']=dr.apply(lambda row: row['whence_y'] if pd.notnull(row['whence_y']) else row['whence_x'],axis=1)
dr['source']=dr.apply(lambda row: row['utm_source_y'] if pd.notnull(row['utm_source_y']) else row['whence'],axis=1)
dr['medium']=dr.apply(lambda row: row['utm_medium_y'] if pd.notnull(row['utm_medium_y']) else row['whence'],axis=1)

#sms = dr.groupby('source').agg({'_id':'count'}).to_excel('sms.xlsx')

dr['source']=dr.apply(lambda row: row['utm_source_y'] if 
                      pd.notnull(row['utm_source_y']) else row['whence'],axis=1)
dr['source']=dr.apply(lambda row: row['sourceGlobal'] if 
                      pd.notnull(row['sourceGlobal']) else row['source'],axis=1)
dr['source']=dr.apply(lambda row: 'Город Тройка' if 
                      row['utm_content']=='gorodtroika' else row['source'],axis=1)
dr['source'] = dr['source'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
dr['source'] = dr['source'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
dr['source'] = dr['source'].apply(lambda x: 'Vbr.ru ссылки' if 
                                  x in ['vbrru', 'upside_vbr','vbrr'] else x)
dr['source'] = dr['source'].apply(lambda x: 'Email' if 
                                 x[:5]=='email' or x[:7]=='TRIGGER' else x)
dr['source'] = dr['source'].apply(lambda x: 'SMS рекламные' if 
                                   x[:3]=='sms' else x)
dr['source'] = dr['source'].apply(lambda x: 'SMS рекламные' if 
                                   x in ['tele2TommorowWON',
                                         'tele2Tommorow',
                                         'tele2FourDays',
                                         'tele2TOP'] or x[:5]=='t2sms' or x =='lostBasket' else x)
dr['source'] = dr['source'].apply(lambda x: 'Admitad' if 
                                   x[:7]=='admitad' else x)
dr['source'] = dr['source'].apply(lambda x: 'Adv.Cake' if 
                                   x[:7]=='advcake' else x)
dr['source'] = dr['source'].apply(lambda x: 'Leadgid' if 
                                   x[:7]=='leadgid' else x)
dr['source'] = dr['source'].apply(lambda x: 'Rubid' if 
                                   x[:5]=='rubid' else x)
dr['source'] = dr['source'].apply(lambda x: 'Cpahub' if 
                                   x[:6]=='cpahub' or x[:6]=='Cpahub' or x== 'get4click'  else x)
dr['source'] = dr['source'].apply(lambda x: 'Перфоманс через Артикс' if 
                                   x[:3]=='ais' else x)
dr['source'] = dr['source'].apply(lambda x: 'Tele2 // Микрозаймы' if 
                                   x[:4]=='tele' else x)
dr['source'] = dr['source'].apply(lambda x: 'Виджеты' if 
                                   x[:10]=='substitute' else x)
dr['source'] = dr['source'].apply(lambda x: 'Браузерные пуши (web)' if 
                                   x[:4]=='push' else x)
dr['source'] = dr['source'].apply(lambda x: 'Exbico' if 
                                   x[:6]=='exbico' else x)
dr['source'] = dr['source'].apply(lambda x: 'Ссылочная масса' if 
                                   x[:8]=='showcase' else x)
dr['source'] = dr['source'].apply(lambda x: 'Статьи на сайте' if 
                                   x[:12]=='content-text' else x)
dr['source'] = dr['source'].apply(lambda x: 'МТС' if 
                                   x[:2]=='mt' else x)
dr['source'] = dr['source'].apply(lambda x: 'Brobank.ru' if 
                                   x[:7]=='brobank' else x)
dr['source'] = dr['source'].apply(lambda x: 'Тестирование гипотез' if 
                                  x[:6]=='yandex' or x[:6]=='google' else x)
dr['source']=dr.apply(lambda row: 'Тестирование гипотез' if 
                      row['utm_source_y']=='yandex' or row['utm_source_y']=='google' else row['source'],axis=1)

dr['source'] = dr.apply(lambda row: 'Металлинвест \\ потребы' if row['productId_x']=='5e9846be79dd27000141f476' 
                        else row['source'], axis=1)
dr['source'] = dr.apply(lambda row: 'Металлинвест \\ потребы' if row['productId_x']=='5ef4473acf22a90001010445' 
                        else row['source'], axis=1)


def get_kibana_reg(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 1000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"REGISTRATION"}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/1000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-1000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID','currentPage',
#                 'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', #'payout', 'status',
#                   'transaction_id', 'clickId', 
                'sourceGlobal']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['dt_reg'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

regs = get_kibana_reg(es, since,till)

#dr['utm_source_x'].value_counts()

regs['currentPage1'] = regs['currentPage'].apply(lambda x: x[:5] if pd.notnull(x) else np.nan)
regs['currentPage2'] = regs['currentPage'].apply(lambda x: x[:9] if pd.notnull(x) else np.nan)

dr=pd.merge(left=dr, right=regs, how='left', on='userId')

# odin = dr[dr['dt_reg'].notnull()]
# odin=odin[odin['product_type']=='LOAN']
# odin['userId'].nunique()

dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Android_mobile' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Huawei_mobile' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Apple_mobile' 
                        else row['source_x'], axis=1)

dr['source_x'] = dr.apply(lambda row: 'Статьи на сайте' if row['currentPage1']=='/news' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Статьи на сайте' if row['currentPage2']=='/articles' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Органический трафик' if row['source_x']=='Ссылочная масса' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'New (2Gis)' if row['source_x']=='2gis' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение 1' if row['source_x']=='Мобильное приложение' 
                        else row['source_x'], axis=1)
# Костыль

dr['status_x'] = dr.apply(lambda x: 'APPROVED' if
                        pd.notnull(x['status_y']) and x['status_y'] in(['APPROVED', 'approved', 'ISSUED']) 
                          else x['status_x'], axis=1)
                          


282179


INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/events/event/_search?scroll=120m [status:200 request:0.989s]


573766; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.308s]


563766; 

...

2686; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.064s]


1686; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.151s]


686; 

In [58]:
dr.head(1)

,_id,type,createdAt,updatedAt,status_x,borrower,partner,product,statusFields,adapterName,whence_x,clickId,id_x,dt,productId_x,product_type,product_name,userId_x,phone,requestId,indexName_x,fallbackResponse_x,json_x,id_y,eventType_x,source_x,params_x,timestamp_x,gaId_x,userId_y,utm_campaign_x,utm_medium_x,utm_source_x,utm_term_x,productType_x,yandexClientID_x,productId_y,whence_y,transitionId,partnerName,payout,status_y,transaction_id,externalId,orderId,transactionId,time_x,date,userId,utm_campaign_y,utm_medium_y,utm_content,utm_source_y,utm_term_y,gaId_y,yandexClientID_y,sourceGlobal_x,score,whence,medium,indexName_y,fallbackResponse_y,json_y,id,eventType_y,source_y,params_y,timestamp_y,gaId,utm_campaign,utm_medium,utm_source,utm_term,productType_y,yandexClientID,currentPage,sourceGlobal_y,time_y,dt_reg,currentPage1,currentPage2
0,5fee3fb2e08dfe0001e15e81,LINK,2020-12-31 21:16:34.929,2020-12-31 21:16:34.929,ACCEPTED,"{'_id': None, 'userId': '5e830b3a78c8e000019e07ad', 'fullName': 'Бажанский Дмитрий Игоревич', 'email': 'bazanskijdmitrij@gmail.com', 'phone': '79012085921'}",Ренессанс Кредит,"{'_id': 5f75c12e3e6c2b0001b0e9eb, 'name': 'Кредитная карта 365', 'productType': 'CREDIT_CARD', 'productName': 'Кредитная карта'}","{'LINK': None, 'NOTE': '2331'}",LEADGID_PARTNER,email_g1806,NaN,5fee3fb2e08dfe0001e15e81,2021-01-01,5f75c12e3e6c2b0001b0e9eb,CREDIT_CARD,Кредитная карта 365,5e830b3a78c8e000019e07ad,79012085921,NaN,NaN,NaN,NaN,NaN,NaN,Email,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT_CARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75,email_g1806,email_g1806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN


In [59]:
dr['dt'] = pd.to_datetime(dr['dt'])
dr['month']  =dr['dt'].dt.strftime('%Y-%m')
dr['Отдел'] = dr['source_x']
dr['Отдел'] = dr['Отдел'].apply(lambda x: 'Direct' if 
                                   x in ['Email','SMS рекламные','Браузерные пуши (web)'] else x)
dr['Отдел'] = dr['Отдел'].apply(lambda x: 'SEO' if 
                                   x in ['Наполнение лендингов по городам',
                                         'Органический трафик','продвижение МП'] else x)
dr['Отдел'] = dr['Отдел'].apply(lambda x: 'SEO' if 
                                   x in ['Наполнение лендингов по городам',
                                         'Органический трафик','продвижение МП',
                                         'накрутка поведенческих факторов',
                                         'Мобильное приложение',
                                         'Новая карточка',
                                         'Новые витрины'] else x)
dr['Отдел'] = dr['Отдел'].apply(lambda x: 'CPA' if 
                                   x in ['Adv.Cake','Admitad','Leadgid','Cpahub','Rubid','Город Тройка',
                                         'Мобильное приложение',
                                         'Металлинвест \ потребы','CPA пиар','Новые продукты \\ дебетовые',
                                         'CallTat','Металлинвест \\ РКО','Промокоды','Чек Скан'] else x)
dr['Отдел'] = dr['Отдел'].apply(lambda x: 'Своя CPA' if 
                                   x in ['Bankiros.ru ссылки','Vbr.ru WL','Vbr.ru ссылки','Brobank.ru',
                                         'Металлинвест','Bankiros.ru WL','Новые партнеры WL'] else x)
dr['Отдел'] = dr['Отдел'].apply(lambda x: 'Контент' if 
                                   x in ['Яндекс.Дзен','Статьи на сайте',
                                         'Рекламные статьи (платные публикации)'] else x)
dr['Отдел'] = dr['Отдел'].apply(lambda x: 'Крупные партнеры' if 
                                   x in ['Tele2 // Микрозаймы','Теле2 \\ Кредитные карты',
                                         'Qiwi \\ Микрозайм','New (2Gis)','Мегафон','Крупный партнер (1)',
                                         'Крупный партнер (2)','Крупный партнер (3)'] else x)
dr['Отдел'] = dr['Отдел'].apply(lambda x: 'КЦ' if 
                                   x in ['Exbico','Внешние базы','Обзвон роботом','Допродажи КЦ'] else x)
dr['Отдел'] = dr['Отдел'].apply(lambda x: 'Перфоманс' if 
                                   x in ['Перфоманс','Тендерное агентство (Артикс)','МТС \\  баннер + СМС',
                                         'Пуш партнерка','Тестирование гипотез'] else x)
dr['Отдел'] = dr['Отдел'].apply(lambda x: 'Продукт' if 
                                   x in ['Быстрая регистрация','Подписка','Конкурс',
                                         'Энвибокс','Новые продукты \\ Подборщик дебетовый карт',
                                         'Новые продукты \\ РКО'] else x)
dr['Отдел'] = dr['Отдел'].apply(lambda x: 'Продукт' if 
                                   x in ['Быстрая регистрация','Подписка','Конкурс',
                                         'Энвибокс','Новые продукты \\ Подборщик дебетовый карт',
                                         'Новые продукты \\ РКО'] else x)
dr['Отдел'] = dr['Отдел'].apply(lambda x: 'Мобильное приложение 1' if 
                                   x in ['Мобильное приложение 1'] else x)

In [60]:
dr_kliki = dr[dr['status_x']=='NEW'].groupby(['Отдел','month','product_type'],as_index=False).agg({'dt':'count'})
dr_kliki['Отдел'].unique()
dr_kliki = dr_kliki[dr_kliki['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено','Контент','КЦ'])]
dr_kliki_pivot = dr_kliki.pivot_table(index=['Отдел','product_type'],
                                      columns='month',
                                      values='dt').sort_index(axis='columns',
                                                                                level='month',
                                                                                ascending=False)
dr_kliki_pivot=dr_kliki_pivot.fillna(0)
dr_kliki_pivot = dr_kliki_pivot.reset_index().copy()
dr_kliki_pivot =dr_kliki_pivot.sort_values('Отдел',ascending=False)
dr_kliki_pivot.to_excel('Ежемесячный отчет/Kliki_new_Alexander_raz.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1AVRkJrGTznlhpezr_2YxxtoyEjunlI_dP97uSCM-wx0')
worksheet = sh.get_worksheet(4) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Ежемесячный отчет/Kliki_new_Alexander_raz.xlsx'
                  )
set_with_dataframe(worksheet, df)
dr_kliki_pivot

month,Отдел,product_type,2021-12,2021-11,2021-10,2021-09,2021-08,2021-07,2021-06,2021-05,2021-04,2021-03,2021-02,2021-01
75,Своя CPA,REFINANCE,2.0,17.0,7.0,4.0,19.0,21.0,28.0,64.0,64.0,60.0,0.0,0.0
74,Своя CPA,LOAN,391.0,2072.0,2878.0,2549.0,2592.0,2980.0,4595.0,7679.0,7989.0,15684.0,12919.0,16979.0
73,Своя CPA,CREDIT_CARD,19.0,30.0,23.0,20.0,58.0,83.0,210.0,340.0,400.0,287.0,318.0,478.0
72,Своя CPA,CASH_LOAN,22.0,59.0,28.0,41.0,47.0,37.0,47.0,112.0,165.0,182.0,51.0,62.0
71,Своя CPA,AUTOCREDIT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
65,Перфоманс,CREDIT_PLEDGE,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0
62,Перфоманс,AUTOCREDIT,0.0,0.0,0.0,0.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
63,Перфоманс,CASH_LOAN,0.0,2.0,9.0,4.0,9.0,9.0,14.0,20.0,13.0,16.0,24.0,9.0
64,Перфоманс,CREDIT_CARD,0.0,2.0,4.0,0.0,4.0,2.0,10.0,10.0,7.0,16.0,11.0,7.0
66,Перфоманс,DEBIT_CARD,0.0,3.0,4.0,3.0,4.0,4.0,19.0,0.0,1.0,6.0,0.0,1.0


In [61]:
dr_kliki = dr[dr['status_x']=='NEW'].groupby(['Отдел','month'],as_index=False).agg({'dt':'count'})
dr_kliki['Отдел'].unique()
dr_kliki = dr_kliki[dr_kliki['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено','Контент','КЦ'])]
dr_kliki_pivot = dr_kliki.pivot_table(index=['Отдел'],
                                      columns='month',
                                      values='dt').sort_index(axis='columns',
                                                                                level='month',
                                                                                ascending=False)
dr_kliki_pivot=dr_kliki_pivot.fillna(0)
dr_kliki_pivot =dr_kliki_pivot.sort_values('Отдел',ascending=False)
dr_kliki_pivot = dr_kliki_pivot.reset_index().copy()
dr_kliki_pivot.to_excel('Ежемесячный отчет/Kliki_new_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1AVRkJrGTznlhpezr_2YxxtoyEjunlI_dP97uSCM-wx0')
worksheet = sh.get_worksheet(5) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Ежемесячный отчет/Kliki_new_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)
dr_kliki_pivot

month,Отдел,2021-12,2021-11,2021-10,2021-09,2021-08,2021-07,2021-06,2021-05,2021-04,2021-03,2021-02,2021-01
0,Своя CPA,434.0,2178.0,2936.0,2614.0,2716.0,3121.0,4880.0,8196.0,8619.0,16213.0,13288.0,17519.0
1,Перфоманс,0.0,65.0,147.0,65.0,91.0,43.0,171.0,154.0,64.0,103.0,43.0,49.0
2,Мобильное приложение 1,1475.0,2065.0,1739.0,1726.0,2083.0,1126.0,1113.0,1505.0,2824.0,830.0,186.0,46.0
3,Крупные партнеры,254.0,2008.0,1388.0,1397.0,1594.0,1820.0,2719.0,1332.0,2530.0,4827.0,898.0,71.0
4,Контент,13.0,23.0,29.0,11.0,12.0,35.0,48.0,66.0,172.0,208.0,28.0,83.0
5,КЦ,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,3.0,2.0,7.0,4.0
6,SEO,714.0,1336.0,1046.0,1110.0,1159.0,1865.0,2040.0,2444.0,3026.0,2548.0,1527.0,1570.0
7,Direct,1258.0,3498.0,2286.0,2773.0,3917.0,5373.0,6279.0,2990.0,4716.0,4902.0,2885.0,3506.0
8,CPA,544.0,2806.0,2726.0,2834.0,2418.0,2570.0,2738.0,3875.0,2931.0,5638.0,5185.0,2952.0


In [62]:
dr_zayavki = dr[dr['status_x']!='NEW'].groupby(['Отдел','month','product_type'],as_index=False).agg({'dt':'count'})
dr_zayavki['Отдел'].unique()
dr_zayavki = dr_zayavki[dr_zayavki['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено','Контент','КЦ'])]
dr_zayavki_pivot = dr_zayavki.pivot_table(index=['Отдел','product_type'],
                                      columns='month',
                                      values='dt').sort_index(axis='columns',
                                                                                level='month',
                                                                                ascending=False)
dr_zayavki_pivot=dr_zayavki_pivot.fillna(0)
dr_zayavki_pivot =dr_zayavki_pivot.sort_values('Отдел',ascending=False)

dr_zayavki_pivot = dr_zayavki_pivot.reset_index().copy()
dr_zayavki_pivot.to_excel('Ежемесячный отчет/Zayavki_Alexander_raz.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1AVRkJrGTznlhpezr_2YxxtoyEjunlI_dP97uSCM-wx0')
worksheet = sh.get_worksheet(6) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Ежемесячный отчет/Zayavki_Alexander_raz.xlsx'
                  )
set_with_dataframe(worksheet, df)
dr_zayavki_pivot

month,Отдел,product_type,2021-12,2021-11,2021-10,2021-09,2021-08,2021-07,2021-06,2021-05,2021-04,2021-03,2021-02,2021-01
0,Своя CPA,REFINANCE,1.0,1.0,2.0,2.0,2.0,7.0,7.0,3.0,65.0,25.0,0.0,0.0
1,Своя CPA,LOAN,50.0,642.0,290.0,647.0,746.0,862.0,385.0,679.0,695.0,1887.0,1373.0,1071.0
2,Своя CPA,CREDIT_CARD,4.0,17.0,8.0,12.0,26.0,21.0,35.0,121.0,188.0,206.0,147.0,135.0
3,Своя CPA,CASH_LOAN,16.0,30.0,12.0,15.0,17.0,21.0,37.0,58.0,76.0,70.0,41.0,46.0
4,Перфоманс,RKO,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Перфоманс,REFINANCE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0
6,Перфоманс,MORTGAGE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,Перфоманс,LOAN,0.0,15.0,36.0,4.0,12.0,6.0,33.0,20.0,11.0,9.0,3.0,3.0
8,Перфоманс,DEBIT_CARD,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,2.0,0.0,0.0
9,Перфоманс,CREDIT_CARD,0.0,1.0,1.0,0.0,5.0,2.0,5.0,0.0,7.0,5.0,3.0,5.0


In [63]:
dr_zayavki = dr[dr['status_x']!='NEW'].groupby(['Отдел','month','product_type'],as_index=False).agg({'dt':'count'})
dr_zayavki['Отдел'].unique()
dr_zayavki = dr_zayavki[dr_zayavki['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено','Контент','КЦ'])]
dr_zayavki_pivot = dr_zayavki.pivot_table(index=['Отдел'],
                                      columns='month',
                                      values='dt',aggfunc='sum').sort_index(axis='columns',
                                                                                level='month',
                                                                                ascending=False)
dr_zayavki_pivot=dr_zayavki_pivot.fillna(0)
dr_zayavki_pivot =dr_zayavki_pivot.sort_values('Отдел',ascending=False)

dr_zayavki_pivot = dr_zayavki_pivot.reset_index().copy()
dr_zayavki_pivot.to_excel('Ежемесячный отчет/Zayavki_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1AVRkJrGTznlhpezr_2YxxtoyEjunlI_dP97uSCM-wx0')
worksheet = sh.get_worksheet(7) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Ежемесячный отчет/Zayavki_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)
dr_zayavki_pivot

month,Отдел,2021-12,2021-11,2021-10,2021-09,2021-08,2021-07,2021-06,2021-05,2021-04,2021-03,2021-02,2021-01
0,Своя CPA,71.0,690.0,312.0,676.0,791.0,911.0,464.0,861.0,1024.0,2188.0,1561.0,1252.0
1,Перфоманс,0.0,17.0,39.0,7.0,23.0,16.0,47.0,34.0,42.0,23.0,27.0,13.0
2,Мобильное приложение 1,384.0,559.0,525.0,481.0,386.0,217.0,172.0,213.0,641.0,202.0,35.0,8.0
3,Крупные партнеры,52.0,427.0,323.0,245.0,381.0,540.0,470.0,163.0,247.0,331.0,72.0,4.0
4,Контент,7.0,15.0,7.0,12.0,5.0,17.0,16.0,11.0,31.0,63.0,22.0,30.0
5,КЦ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,SEO,149.0,283.0,284.0,223.0,263.0,316.0,514.0,522.0,743.0,680.0,497.0,327.0
7,Direct,122.0,351.0,259.0,246.0,398.0,451.0,637.0,307.0,530.0,646.0,453.0,663.0
8,CPA,216.0,533.0,431.0,405.0,252.0,351.0,578.0,1060.0,545.0,1135.0,1260.0,785.0


In [64]:
cr_klik_zayavka = dr_zayavki.merge(dr_kliki,how='left',on=['Отдел','month'])
cr_klik_zayavka
cr_klik_zayavka['cr_klik_zayavka_'] = ((cr_klik_zayavka['dt_x']/cr_klik_zayavka['dt_y'])*100).round(2)
cr_klik_zayavka_pivot =cr_klik_zayavka.pivot_table(index=['Отдел'],
                                                   columns='month',
                                                   values='cr_klik_zayavka_').sort_index(axis='columns',
                                                                                level='month',
                                                                                ascending=False)
cr_klik_zayavka_pivot = cr_klik_zayavka_pivot.fillna(0)
cr_klik_zayavka_pivot =cr_klik_zayavka_pivot.sort_values('Отдел',ascending=False)
cr_klik_zayavka_pivot = cr_klik_zayavka_pivot.reset_index().copy()

cr_klik_zayavka_pivot.to_excel('Ежемесячный отчет/cr_kliki_Zayavki_Alexander_raz.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1AVRkJrGTznlhpezr_2YxxtoyEjunlI_dP97uSCM-wx0')
worksheet = sh.get_worksheet(8) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Ежемесячный отчет/cr_kliki_Zayavki_Alexander_raz.xlsx'
                  )
set_with_dataframe(worksheet, df)
cr_klik_zayavka_pivot

month,Отдел,2021-12,2021-11,2021-10,2021-09,2021-08,2021-07,2021-06,2021-05,2021-04,2021-03,2021-02,2021-01
0,Своя CPA,4.090000,7.922500,2.657500,6.465000,7.282500,7.295000,2.377500,2.627500,2.967500,3.372500,3.916667,2.380000
1,Перфоманс,0.000000,8.720000,8.843333,5.385000,6.317500,7.442000,9.160000,7.360000,13.124000,4.466000,15.700000,8.840000
2,Мобильное приложение 1,5.206000,3.385000,5.035000,3.981429,3.706000,2.754286,1.931250,2.022857,2.836250,3.477143,4.705000,5.800000
3,Крупные партнеры,20.470000,10.635000,23.270000,4.382500,11.955000,9.890000,5.763333,4.083333,3.256667,2.286667,8.020000,1.880000
4,Контент,17.946667,21.740000,12.065000,27.272500,13.886667,16.190000,11.110000,5.560000,4.502500,6.058000,15.712000,9.035000
5,КЦ,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25.000000
6,SEO,2.980000,3.024286,3.878571,2.870000,3.781667,2.117500,2.798889,2.373333,3.070000,2.964444,4.067500,2.605000
7,Direct,1.385714,2.006000,1.886667,1.478333,2.034000,1.400000,2.537500,2.565000,1.403750,2.198333,3.142000,3.151667
8,CPA,9.927500,4.747500,3.950000,2.860000,2.602500,2.730000,5.277500,5.472000,3.100000,2.875714,4.051667,4.433333


In [65]:
dr_vidachy = dr[dr['status_x'].isin(['APPROVED',
                                     'ISSUED'])].groupby(['Отдел',
                                                                     'month',
                                                                     'product_type'],as_index=False).agg({'dt':'count'})
dr_vidachy['Отдел'].unique()
dr_vidachy = dr_vidachy[dr_vidachy['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено','Контент','КЦ'])]
dr_vidachy_pivot = dr_vidachy.pivot_table(index=['Отдел','product_type'],
                                      columns='month',
                                      values='dt',aggfunc='sum').sort_index(axis='columns',
                                                                                level='month',
                                                                                ascending=False)
dr_vidachy_pivot=dr_vidachy_pivot.fillna(0)
dr_vidachy_pivot =dr_vidachy_pivot.sort_values('Отдел',ascending=False)

dr_vidachy_pivot = dr_vidachy_pivot.reset_index().copy()
dr_vidachy_pivot.to_excel('Ежемесячный отчет/Vidachy_Alexander_raz.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1AVRkJrGTznlhpezr_2YxxtoyEjunlI_dP97uSCM-wx0')
worksheet = sh.get_worksheet(9) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Ежемесячный отчет/Vidachy_Alexander_raz.xlsx'
                  )
set_with_dataframe(worksheet, df)
dr_vidachy_pivot

month,Отдел,product_type,2021-12,2021-11,2021-10,2021-09,2021-08,2021-07,2021-06,2021-05,2021-04,2021-03,2021-02,2021-01
0,Своя CPA,REFINANCE,0.0,1.0,2.0,2.0,2.0,7.0,7.0,3.0,1.0,2.0,0.0,0.0
1,Своя CPA,LOAN,4.0,28.0,40.0,37.0,48.0,62.0,67.0,100.0,151.0,456.0,394.0,352.0
2,Своя CPA,CREDIT_CARD,0.0,0.0,0.0,0.0,1.0,0.0,1.0,7.0,3.0,5.0,2.0,11.0
3,Своя CPA,CASH_LOAN,0.0,12.0,4.0,8.0,9.0,15.0,6.0,8.0,21.0,17.0,7.0,17.0
4,Перфоманс,LOAN,0.0,3.0,17.0,3.0,2.0,0.0,13.0,2.0,5.0,3.0,0.0,0.0
5,Перфоманс,CREDIT_CARD,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
6,Перфоманс,CASH_LOAN,0.0,1.0,1.0,0.0,0.0,2.0,1.0,2.0,5.0,3.0,3.0,0.0
7,Мобильное приложение 1,CREDIT_PLEDGE,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
8,Мобильное приложение 1,AUTOCREDIT,0.0,0.0,1.0,2.0,3.0,1.0,1.0,3.0,0.0,0.0,0.0,0.0
9,Мобильное приложение 1,CASH_LOAN,0.0,51.0,73.0,70.0,78.0,56.0,0.0,3.0,28.0,34.0,2.0,2.0


In [66]:
dr_vidachy = dr[dr['status_x'].isin(['APPROVED',
                                     'ISSUED'])].groupby(['Отдел','product_type',
                                                                     'month'],as_index=False).agg({'dt':'count'})
dr_vidachy['Отдел'].unique()
dr_vidachy = dr_vidachy[dr_vidachy['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено','Контент','КЦ'])]
dr_vidachy_pivot = dr_vidachy.pivot_table(index=['Отдел'],
                                      columns='month',
                                      values='dt',aggfunc='sum').sort_index(axis='columns',
                                                                                level='month',
                                                                                ascending=False)
dr_vidachy_pivot=dr_vidachy_pivot.fillna(0)
dr_vidachy_pivot=dr_vidachy_pivot.sort_values('Отдел',ascending=False)
dr_vidachy_pivot = dr_vidachy_pivot.reset_index().copy()
dr_vidachy_pivot.to_excel('Ежемесячный отчет/Vidachy_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1AVRkJrGTznlhpezr_2YxxtoyEjunlI_dP97uSCM-wx0')
worksheet = sh.get_worksheet(10) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Ежемесячный отчет/Vidachy_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)
dr_vidachy_pivot

month,Отдел,2021-12,2021-11,2021-10,2021-09,2021-08,2021-07,2021-06,2021-05,2021-04,2021-03,2021-02,2021-01
0,Своя CPA,4.0,41.0,46.0,47.0,60.0,84.0,81.0,118.0,176.0,480.0,403.0,380.0
1,Перфоманс,0.0,4.0,18.0,3.0,2.0,3.0,15.0,4.0,11.0,6.0,3.0,0.0
2,Мобильное приложение 1,14.0,70.0,93.0,98.0,106.0,72.0,23.0,18.0,33.0,36.0,2.0,2.0
3,Крупные партнеры,4.0,62.0,67.0,52.0,58.0,119.0,84.0,38.0,54.0,99.0,34.0,3.0
4,Контент,0.0,1.0,3.0,4.0,2.0,5.0,5.0,0.0,2.0,7.0,5.0,2.0
5,SEO,3.0,53.0,67.0,55.0,83.0,114.0,125.0,75.0,70.0,89.0,53.0,37.0
6,Direct,9.0,49.0,116.0,96.0,117.0,210.0,233.0,70.0,106.0,140.0,35.0,63.0
7,CPA,9.0,97.0,55.0,125.0,72.0,76.0,65.0,77.0,43.0,84.0,46.0,72.0


In [67]:
dr_vidachy


,Отдел,product_type,month,dt
7,CPA,AUTOCREDIT,2021-05,1
8,CPA,CASH_LOAN,2021-01,40
9,CPA,CASH_LOAN,2021-02,27
10,CPA,CASH_LOAN,2021-03,43
11,CPA,CASH_LOAN,2021-04,19
12,CPA,CASH_LOAN,2021-05,14
13,CPA,CASH_LOAN,2021-06,9
14,CPA,CASH_LOAN,2021-07,37
15,CPA,CASH_LOAN,2021-08,47
16,CPA,CASH_LOAN,2021-09,56


In [68]:
cr_zayavka_vidacha = dr_zayavki.merge(dr_vidachy,how='left',on=['Отдел','month','product_type'])
cr_zayavka_vidacha['cr_zayavka_vidacha_'] = ((cr_zayavka_vidacha['dt_y']/cr_zayavka_vidacha['dt_x'])*100).round(2)
cr_zayavka_vidacha_pivot = cr_zayavka_vidacha.pivot_table(index=['Отдел',
                                                                'product_type'],
                                                                 columns='month',
                                                                 values='cr_zayavka_vidacha_').sort_index(axis='columns',
                                                                                level='month',
                                                                                ascending=False)
cr_zayavka_vidacha_pivot = cr_zayavka_vidacha_pivot.fillna(0)
cr_zayavka_vidacha_pivot=cr_zayavka_vidacha_pivot.sort_values('Отдел',ascending=False)
cr_zayavka_vidacha_pivot = cr_zayavka_vidacha_pivot.reset_index().copy()
cr_zayavka_vidacha_pivot.to_excel('Ежемесячный отчет/cr_zayavki_Vidachy_Alexander_raz.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1AVRkJrGTznlhpezr_2YxxtoyEjunlI_dP97uSCM-wx0')
worksheet = sh.get_worksheet(11) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Ежемесячный отчет/cr_zayavki_Vidachy_Alexander_raz.xlsx'
                  )
set_with_dataframe(worksheet, df)
cr_zayavka_vidacha_pivot

month,Отдел,product_type,2021-12,2021-11,2021-10,2021-09,2021-08,2021-07,2021-06,2021-05,2021-04,2021-03,2021-02,2021-01
0,Своя CPA,REFINANCE,0.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,1.54,8.00,0.00,0.00
1,Своя CPA,LOAN,8.00,4.36,13.79,5.72,6.43,7.19,17.40,14.73,21.73,24.17,28.70,32.87
2,Своя CPA,CREDIT_CARD,0.00,0.00,0.00,0.00,3.85,0.00,2.86,5.79,1.60,2.43,1.36,8.15
3,Своя CPA,CASH_LOAN,0.00,40.00,33.33,53.33,52.94,71.43,16.22,13.79,27.63,24.29,17.07,36.96
4,Перфоманс,LOAN,0.00,20.00,47.22,75.00,16.67,0.00,39.39,10.00,45.45,33.33,0.00,0.00
5,Перфоманс,CREDIT_CARD,0.00,0.00,0.00,0.00,0.00,50.00,20.00,0.00,14.29,0.00,0.00,0.00
6,Перфоманс,CASH_LOAN,0.00,100.00,50.00,0.00,0.00,33.33,11.11,15.38,23.81,50.00,15.00,0.00
7,Мобильное приложение 1,CREDIT_PLEDGE,0.00,0.00,0.00,0.00,0.00,0.00,100.00,100.00,0.00,0.00,0.00,0.00
8,Мобильное приложение 1,AUTOCREDIT,0.00,0.00,100.00,100.00,100.00,100.00,100.00,100.00,0.00,0.00,0.00,0.00
9,Мобильное приложение 1,CASH_LOAN,0.00,21.98,31.33,39.11,45.61,49.56,0.00,2.97,7.51,26.15,9.09,50.00


In [69]:
cr_zayavka_vidacha = dr_zayavki.merge(dr_vidachy,how='left',on=['Отдел','month','product_type'])
cr_zayavka_vidacha['cr_zayavka_vidacha_'] = ((cr_zayavka_vidacha['dt_y']/cr_zayavka_vidacha['dt_x'])*100).round(2)
cr_zayavka_vidacha_pivot = cr_zayavka_vidacha.pivot_table(index=['Отдел'
                                                            ],
                                                                 columns='month',
                                                                 values='cr_zayavka_vidacha_').sort_index(axis='columns',
                                                                                level='month',
                                                                                ascending=False)
cr_zayavka_vidacha_pivot = cr_zayavka_vidacha_pivot.fillna(0)
cr_zayavka_vidacha_pivot=cr_zayavka_vidacha_pivot.sort_values('Отдел',ascending=False)
cr_zayavka_vidacha_pivot = cr_zayavka_vidacha_pivot.reset_index().copy()
cr_zayavka_vidacha_pivot.to_excel('Ежемесячный отчет/cr_zayavki_Vidachy_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1AVRkJrGTznlhpezr_2YxxtoyEjunlI_dP97uSCM-wx0')
worksheet = sh.get_worksheet(12) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Ежемесячный отчет/cr_zayavki_Vidachy_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)
cr_zayavka_vidacha_pivot

month,Отдел,2021-12,2021-11,2021-10,2021-09,2021-08,2021-07,2021-06,2021-05,2021-04,2021-03,2021-02,2021-01
0,Своя CPA,8.000000,48.1200,49.0400,53.016667,40.805,59.540000,34.120000,33.577500,13.125000,14.722500,15.7100,25.993333
1,Перфоманс,0.000000,60.0000,48.6100,75.000000,16.670,41.665000,23.500000,12.690000,27.850000,41.665000,15.0000,0.000000
2,Мобильное приложение 1,7.720000,23.5425,48.9320,51.018000,54.042,53.970000,64.828000,52.471667,18.197500,11.633333,9.0900,50.000000
3,Крупные партнеры,7.690000,14.5900,20.7400,60.535000,57.390,19.520000,25.633333,44.800000,35.950000,29.345000,47.2200,100.000000
4,Контент,0.000000,16.6700,75.0000,71.430000,66.665,34.820000,36.905000,0.000000,21.665000,18.230000,26.2500,8.125000
5,SEO,2.953333,47.4925,51.8380,45.951667,54.478,63.341667,56.623333,54.843333,16.000000,15.748333,12.0250,15.228000
6,Direct,16.070000,35.4525,55.0000,50.760000,32.010,60.238000,31.790000,36.365000,15.975000,20.332000,8.4175,17.283333
7,CPA,7.640000,28.7350,32.5825,42.830000,37.645,49.740000,35.237500,43.784000,13.443333,17.090000,5.6850,12.443333


In [70]:
since = date(2021,1,4)
till=date(2021,12,31)

In [71]:
df = get_exact_claims(since, till, client)
# df['requestId'] = df[df['partner']=='MoneyMan']['statusFields'].apply(mm_requestId)
dp = get_kibana_postback(es, since)
def float_apply(x):
    try:
        return float(x)
    except ValueError:
        return None
dp['payout'] = dp['payout'].apply(float_apply)
dp['payout'] = dp.apply(lambda x: x['payout']*1.2 if x['partnerName']=='admitad' else x['payout'], axis=1)
dp = dp[dp['status'].isin(['APPROVED', 'approved', 'ISSUED'])].sort_values('payout').drop_duplicates('transitionId', keep='last')
dr = pd.merge(left=df, right=dp, left_on='id', right_on='transitionId', how='left')
dc = get_kibana_claims(es, since,till)[['clickId', 'userId', 'utm_campaign', 'utm_medium', 
                                        'utm_source', 'utm_term','utm_content','gaId', 'yandexClientID', 'sourceGlobal']]
# print(dr.columns)
dr = pd.merge(left=dr, right=dc[dc['clickId'].notnull()].drop_duplicates('clickId'), on='clickId', how='left')
# print(dr.columns)

cre = cre_get(es, since)
dr['score'] = dr['userId_x'].map(cre).fillna(0).astype(int)
dr['whence_source'] = dr['whence_x'].apply(lambda x: x.split("_")[0] if x.find('_')>-1 and x.find('upside')<0 else x)
#dr['payout'] = dr.apply(lambda x: 1445 if x['partner']=='MigONE' and pd.isnull(x['payout']) else x['payout'], axis=1)
dr['payout'] = dr.apply(lambda x: 3200 if x['partner']=='MoneyMan' and pd.isnull(x['payout']) else x['payout'], axis=1)
# dr['payout'] = dr.apply(lambda x: 3800 if x['partner']=='Альфа-Банк' and pd.isnull(x['payout']) else x['payout'], axis=1)
# dr['payout'] = dr.apply(lambda x: 3000 if x['partner']=='Тинькофф Банк' and pd.isnull(x['payout']) else x['payout'], axis=1)
# dr['payout'] = dr.apply(lambda x: 3437 if x['partner']=='МигКредит' and pd.isnull(x['payout']) else x['payout'], axis=1)
# dr['payout'] = dr.apply(lambda x: 700 if x['partner']=='CarMoney' and pd.isnull(x['payout']) else x['payout'], axis=1)

dreg = get_registrations(dr['userId_x'].tolist())
regs = dreg['userId'].tolist()
dr['registration'] = dr['userId_x'].apply(lambda x: x if x in regs else np.nan)
dreg.index = dreg['userId']
reg_time = dreg['reg_time'].to_dict()
dreg = dreg[dreg['utm_source'].notnull()]['utm_source'].to_dict()

dr['Целевое действие'] = dr['productId_x'].apply(lambda x: 'Заявка' if x in claim_offers else 'Выдача')

dr['source'] = dr.apply(lambda x: x['sourceGlobal'] if pd.notnull(x['sourceGlobal']) 
         else (x['utm_source_y'] if pd.notnull(x['utm_source_y']) 
         else x['whence_source']),axis=1).fillna('Direct&organic')
dr['source_type'] = dr.apply(lambda x:'WhiteLabel' if pd.notnull(x['sourceGlobal']) 
         else 'utm_source_y' if pd.notnull(x['utm_source_y']) 
         else 'whence_source',axis=1).fillna('other')
# dr.fillna('').groupby(['utm_source_y', 'sourceGlobal', 'whence_source']).count()
dr['source'] = dr['source'].apply(lambda x: x.replace('test_partner', 'tele2')\
.replace('tele_2', 'tele2'))#.replace('showcase', 'Direct&organic'))

dr['source_ext'] = dr.apply(lambda x: f"{x['source']} ({dreg[x['userId_x']]})" 
         if x['userId_x'] in dreg.keys() and  x['source'] in ['email']
                            else x['source'],axis=1)
dr['reg_time'] = dr['userId_x'].map(reg_time)
dr[(dr['status_x'].isin(['APPROVED', 'ISSUED']))].groupby(['partner']).agg({'_id':'count',
                             'payout': 'sum'}).to_excel('reports/part.xlsx')

dr['source'] = dr['source_ext']

# dr['source']=dr.apply(lambda row: row['utm_source_y'] if 
#                       pd.notnull(row['utm_source_y']) else row['whence_source'],axis=1)
# dr['source']=dr.apply(lambda row: row['sourceGlobal'] if 
#                       pd.notnull(row['sourceGlobal']) else row['whence_source'],axis=1)
# dr['source']=dr.apply(lambda row: 'Город Тройка' if 
#                       row['utm_content_y']=='gorodtroika' else row['whence_source'],axis=1)
dr['source'] = dr['source'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
dr['source'] = dr['source'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
dr['source'] = dr['source'].apply(lambda x: 'Vbr.ru ссылки' if 
                                  x in ['vbrru', 'upside_vbr','vbrr'] else x)
dr['source'] = dr['source'].apply(lambda x: 'Email' if 
                                 x[:5]=='email' or x[:7]=='TRIGGER' else x)
dr['source'] = dr['source'].apply(lambda x: 'SMS рекламные' if 
                                   x[:3]=='sms' else x)
dr['source'] = dr['source'].apply(lambda x: 'SMS рекламные' if 
                                   x in ['tele2TommorowWON',
                                         'tele2Tommorow',
                                         'tele2FourDays',
                                         'tele2TOP'] or x[:5]=='t2sms' or x =='lostBasket' else x)
dr['source'] = dr['source'].apply(lambda x: 'Admitad' if 
                                   x[:7]=='admitad' else x)
dr['source'] = dr['source'].apply(lambda x: 'Adv.Cake' if 
                                   x[:7]=='advcake' else x)
dr['source'] = dr['source'].apply(lambda x: 'Leadgid' if 
                                   x[:7]=='leadgid' else x)
dr['source'] = dr['source'].apply(lambda x: 'Rubid' if 
                                   x[:5]=='rubid' else x)
dr['source'] = dr['source'].apply(lambda x: 'Cpahub' if 
                                   x[:6]=='cpahub' or x[:6]=='Cpahub' or x== 'get4click'  else x)
dr['source'] = dr['source'].apply(lambda x: 'Перфоманс через Артикс' if 
                                   x[:3]=='ais' else x)
dr['source'] = dr['source'].apply(lambda x: 'Tele2 // Микрозаймы' if 
                                   x[:4]=='tele' else x)
dr['source'] = dr['source'].apply(lambda x: 'Теле2 \\ Кредитные карты' if 
                                   x[:5]=='t2_kk' else x)
dr['source'] = dr['source'].apply(lambda x: 'Виджеты' if 
                                   x[:10]=='substitute' else x)
dr['source'] = dr['source'].apply(lambda x: 'Браузерные пуши (web)' if 
                                   x[:4]=='push' else x)
dr['source'] = dr['source'].apply(lambda x: 'Exbico' if 
                                   x[:6]=='exbico' else x)
dr['source'] = dr['source'].apply(lambda x: 'Ссылочная масса' if 
                                   x[:8]=='showcase' else x)
dr['source'] = dr['source'].apply(lambda x: 'Статьи на сайте' if 
                                   x[:12]=='content-text' else x)
dr['source'] = dr['source'].apply(lambda x: 'МТС' if 
                                   x[:2]=='mt' else x)
dr['source'] = dr['source'].apply(lambda x: 'Brobank.ru' if 
                                   x[:7]=='brobank' else x)
dr['source'] = dr['source'].apply(lambda x: 'Тестирование гипотез' if 
                                  x[:6]=='yandex' or x[:6]=='google' else x)
dr['source']=dr.apply(lambda row: 'Тестирование гипотез' if 
                      row['utm_source_y']=='yandex' or row['utm_source_y']=='google' else row['source'],axis=1)

dr['source'] = dr.apply(lambda row: 'Металлинвест \\ потребы' if row['productId_x']=='5e9846be79dd27000141f476' 
                        else row['source'], axis=1)
dr['source'] = dr.apply(lambda row: 'Металлинвест \\ потребы' if row['productId_x']=='5ef4473acf22a90001010445' 
                        else row['source'], axis=1)
dr['status_x'] = dr.apply(lambda x: 'APPROVED' if
                        pd.notnull(x['status_y']) and x['status_y'] in(['APPROVED', 'approved', 'ISSUED']) 
                          else x['status_x'], axis=1)


def get_kibana_reg(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 1000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"REGISTRATION"}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/1000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-1000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID','currentPage',
#                 'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', #'payout', 'status',
#                   'transaction_id', 'clickId', 
                'sourceGlobal']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['dt_reg'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

regs = get_kibana_reg(es, since,till)

dr['utm_source_x'].value_counts()

regs['currentPage1'] = regs['currentPage'].apply(lambda x: x[:5] if pd.notnull(x) else np.nan)
regs['currentPage2'] = regs['currentPage'].apply(lambda x: x[:9] if pd.notnull(x) else np.nan)

dr=pd.merge(left=dr, right=regs, how='left', on='userId')

# odin = dr[dr['dt_reg'].notnull()]
# odin=odin[odin['product_type']=='LOAN']
# odin['userId'].nunique()

dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Android_mobile' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Huawei_mobile' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Apple_mobile' 
                        else row['source_x'], axis=1)

dr['source_x'] = dr.apply(lambda row: 'Статьи на сайте' if row['currentPage1']=='/news' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Статьи на сайте' if row['currentPage2']=='/articles' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Органический трафик' if row['source_x']=='Ссылочная масса' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'New (2Gis)' if row['source_x']=='2gis' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение 1' if row['source_x']=='Мобильное приложение' 
                        else row['source_x'], axis=1)
dr['month']=(pd.to_datetime(dr['dt']).dt.strftime('%Y-%m'))
# fields = ['source_x','week']
# # d0 = tele2_metrika(since, till, '')
# d1 = dr.groupby((fields),as_index=False).agg({'_id': 'count'})
# d2 = dr[(dr['status_x']!='NEW')
#    ].groupby((fields),as_index=False).agg({'_id': 'count'})
d3 = dr[(dr['status_x'].isin(['APPROVED', 'ISSUED']))
   ].groupby(['source_x','month','product_type'],as_index=False).agg({'_id': 'count', 'payout': 'sum'})

# dd1 = pd.merge(d0, d1, left_index=True, right_index=True, how='right')
# dd2 = pd.merge(d1, d2, left_index=True, right_index=True, how='left')
# dr_group = pd.merge(dd2, d3, left_index=True, right_index=True, how='left')
d3.sort_index(inplace = True, ascending=False)

280319


INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/events/event/_search?scroll=120m [status:200 request:7.396s]


572228; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.985s]


562228; 

...

2495; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.313s]


1495; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.170s]


495; 

In [72]:

d3['Отдел'] = d3['source_x']
d3['Отдел'] = d3['Отдел'].apply(lambda x: 'Direct' if 
                                   x in ['Email','SMS рекламные','Браузерные пуши (web)'] else x)
d3['Отдел'] = d3['Отдел'].apply(lambda x: 'SEO' if 
                                   x in ['Наполнение лендингов по городам',
                                         'Органический трафик','продвижение МП'] else x)
d3['Отдел'] = d3['Отдел'].apply(lambda x: 'SEO' if 
                                   x in ['Наполнение лендингов по городам',
                                         'Органический трафик','продвижение МП',
                                         'накрутка поведенческих факторов',
                                         'Мобильное приложение',
                                         'Новая карточка',
                                         'Новые витрины'] else x)
d3['Отдел'] = d3['Отдел'].apply(lambda x: 'CPA' if 
                                   x in ['Adv.Cake','Admitad','Leadgid','Cpahub','Rubid','Город Тройка',
                                         'Мобильное приложение',
                                         'Металлинвест \ потребы','CPA пиар','Новые продукты \\ дебетовые',
                                         'CallTat','Металлинвест \\ РКО','Промокоды','Чек Скан'] else x)
d3['Отдел'] = d3['Отдел'].apply(lambda x: 'Своя CPA' if 
                                   x in ['Bankiros.ru ссылки','Vbr.ru WL','Vbr.ru ссылки','Brobank.ru',
                                         'Металлинвест','Bankiros.ru WL','Новые партнеры WL'] else x)
d3['Отдел'] = d3['Отдел'].apply(lambda x: 'Контент' if 
                                   x in ['Яндекс.Дзен','Статьи на сайте',
                                         'Рекламные статьи (платные публикации)'] else x)
d3['Отдел'] = d3['Отдел'].apply(lambda x: 'Крупные партнеры' if 
                                   x in ['Tele2 // Микрозаймы','Теле2 \\ Кредитные карты',
                                         'Qiwi \\ Микрозайм','New (2Gis)','Мегафон','Крупный партнер (1)',
                                         'Крупный партнер (2)','Крупный партнер (3)'] else x)
d3['Отдел'] = d3['Отдел'].apply(lambda x: 'КЦ' if 
                                   x in ['Exbico','Внешние базы','Обзвон роботом','Допродажи КЦ'] else x)
d3['Отдел'] = d3['Отдел'].apply(lambda x: 'Перфоманс' if 
                                   x in ['Перфоманс','Тендерное агентство (Артикс)','МТС \\  баннер + СМС',
                                         'Пуш партнерка','Тестирование гипотез'] else x)
d3['Отдел'] = d3['Отдел'].apply(lambda x: 'Продукт' if 
                                   x in ['Быстрая регистрация','Подписка','Конкурс',
                                         'Энвибокс','Новые продукты \\ Подборщик дебетовый карт',
                                         'Новые продукты \\ РКО'] else x)
d3['Отдел'] = d3['Отдел'].apply(lambda x: 'Продукт' if 
                                   x in ['Быстрая регистрация','Подписка','Конкурс',
                                         'Энвибокс','Новые продукты \\ Подборщик дебетовый карт',
                                         'Новые продукты \\ РКО'] else x)
d3['Отдел'] = d3['Отдел'].apply(lambda x: 'Мобильное приложение 1' if 
                                   x in ['Мобильное приложение 1'] else x)

In [73]:
d3['payout']=d3['payout'].astype('int')

In [74]:
d3

,source_x,month,product_type,_id,payout,Отдел
455,Тестирование гипотез,2021-11,LOAN,4,3695,Перфоманс
454,Тестирование гипотез,2021-11,CASH_LOAN,1,300,Перфоманс
453,Тестирование гипотез,2021-10,LOAN,26,5946,Перфоманс
452,Тестирование гипотез,2021-10,CASH_LOAN,1,300,Перфоманс
451,Тестирование гипотез,2021-09,LOAN,3,225,Перфоманс
450,Тестирование гипотез,2021-08,LOAN,2,1460,Перфоманс
449,Тестирование гипотез,2021-07,CREDIT_CARD,1,810,Перфоманс
448,Тестирование гипотез,2021-07,CASH_LOAN,2,430,Перфоманс
447,Тестирование гипотез,2021-06,LOAN,13,4645,Перфоманс
446,Тестирование гипотез,2021-06,CREDIT_CARD,1,810,Перфоманс


In [75]:
d3_pivot = d3[d3['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено',
                                'Контент','КЦ'])].pivot_table(index=['Отдел','product_type'],
                                                              columns='month',
                                                              values='payout',aggfunc='sum').sort_index(axis='columns',
                                                                                level='month',
                                                                                ascending=False)
d3_pivot=d3_pivot.fillna(0)
d3_pivot=d3_pivot.sort_values('Отдел',ascending=False)

d3_pivot.to_excel('Ежемесячный отчет/virucka_Alexander_raz.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1AVRkJrGTznlhpezr_2YxxtoyEjunlI_dP97uSCM-wx0')
worksheet = sh.get_worksheet(13) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Ежемесячный отчет/virucka_Alexander_raz.xlsx'
                  )
set_with_dataframe(worksheet, df)
d3_pivot

month                                 2021-12   2021-11   2021-10   2021-09   2021-08   2021-07   2021-06   2021-05   2021-04   2021-03   2021-02   2021-01
Отдел                  product_type                                                                                                                        
Своя CPA               REFINANCE          0.0     300.0     600.0     600.0     600.0    2100.0    2100.0     900.0     150.0     300.0       0.0       0.0
                       LOAN           14070.0  101435.0  140755.0  136810.0  151020.0  194370.0  127745.0  164525.0  157622.0  459821.0  309391.0  289701.0
                       CREDIT_CARD        0.0       0.0       0.0       0.0    3000.0       0.0    2300.0   11110.0    4000.0   11300.0    2330.0   17832.0
                       CASH_LOAN          0.0    3600.0    1200.0    2030.0    1690.0    3010.0     900.0    1200.0    3110.0    2400.0    1010.0    2300.0
Перфоманс              LOAN               0.0    3695.0    5946.0     225.0    1460.0       0.0    4645.0    2397.0    7750.0    2470.0       0.0       0.0
                       CREDIT_CARD        0.0       0.0       0.0       0.0       0.0     810.0     810.0       0.0    2000.0       0.0       0.0       0.0
                       CASH_LOAN          0.0     300.0     300.0       0.0       0.0     430.0    7000.0     300.0     710.0     410.0     430.0       0.0
Мобильное приложение 1 CREDIT_PLEDGE      0.0       0.0       0.0       0.0       0.0       0.0     300.0     300.0       0.0       0.0       0.0       0.0
                       AUTOCREDIT         0.0       0.0     300.0     600.0     900.0     300.0     300.0     900.0       0.0       0.0       0.0       0.0
                       CASH_LOAN          0.0   19450.0   24400.0   17670.0   19630.0   13060.0       0.0   14050.0   24350.0    4420.0     280.0     280.0
                       CREDIT_CARD    11520.0   25400.0   20530.0   19480.0   25140.0    1620.0    7710.0    6726.0       0.0    1844.0       0.0       0.0
                       DEBIT_CARD         0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0     800.0       0.0       0.0       0.0
                       LOAN           25420.0    8124.0       0.0    4400.0   11680.0    1495.0    8019.0    1097.0    4690.0     310.0       0.0       0.0
                       REFINANCE          0.0    2400.0    2700.0    5100.0    3000.0    2700.0    3300.0    1800.0    6700.0       0.0       0.0       0.0
Крупные партнеры       REFINANCE          0.0       0.0       0.0     300.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0
                       LOAN            7221.0  125963.0  137099.0  141739.0  143140.0  221510.0  138335.0   60700.0  141034.0  154230.0   50501.0     435.0
                       CREDIT_CARD        0.0       0.0    8800.0       0.0    2610.0    4800.0       0.0       0.0    2800.0       0.0       0.0       0.0
                       CASH_LOAN          0.0       0.0       0.0       0.0     800.0    1840.0       0.0     300.0    4030.0     300.0       0.0     150.0
Контент                LOAN               0.0       0.0       0.0       0.0       0.0    2800.0     300.0       0.0       0.0    3682.0    8360.0    1620.0
                       REFINANCE          0.0       0.0       0.0     300.0     300.0       0.0       0.0       0.0     150.0       0.0       0.0       0.0
                       CASH_LOAN          0.0     300.0     900.0     560.0     200.0     860.0     600.0       0.0     130.0     540.0    6900.0     140.0
SEO                    LOAN            4000.0    4512.0   10393.0    7744.0   21064.0   14804.0   31879.0    8980.0   26429.0   37810.0   30985.0   12171.0
                       DEBIT_CARD         0.0       0.0       0.0    1000.0       0.0       0.0       0.0       0.0     338.0    1667.0       0.0    1020.0
                       CREDIT_PLEDGE      0.0       0.0       0.0       0.0       0.0     600.0    3

In [76]:
d3_pivot = d3[d3['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено',
                                'Контент','КЦ'])].pivot_table(index=['Отдел'],
                                                              columns='month',
                                                              values='payout',aggfunc='sum').sort_index(axis='columns',
                                                                                level='month',
                                                                                ascending=False)
d3_pivot=d3_pivot.fillna(0)
d3_pivot=d3_pivot.sort_values('Отдел',ascending=False)
d3_pivot.to_excel('Ежемесячный отчет/virucka_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1AVRkJrGTznlhpezr_2YxxtoyEjunlI_dP97uSCM-wx0')
worksheet = sh.get_worksheet(14) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Ежемесячный отчет/virucka_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)
d3_pivot

month,2021-12,2021-11,2021-10,2021-09,2021-08,2021-07,2021-06,2021-05,2021-04,2021-03,2021-02,2021-01
Отдел,,,,,,,,,,,,
Своя CPA,14070.0,105335.0,142555.0,139440.0,156310.0,199480.0,133045.0,177735.0,164882.0,473821.0,312731.0,309833.0
Перфоманс,0.0,3995.0,6246.0,225.0,1460.0,1240.0,12455.0,2697.0,10460.0,2880.0,430.0,0.0
Мобильное приложение 1,36940.0,55374.0,47930.0,47250.0,60350.0,19175.0,19629.0,24873.0,36540.0,6574.0,280.0,280.0
Крупные партнеры,7221.0,125963.0,145899.0,142039.0,146550.0,228150.0,138335.0,61000.0,147864.0,154530.0,50501.0,585.0
Контент,0.0,300.0,900.0,860.0,500.0,3660.0,900.0,0.0,280.0,4222.0,15260.0,1760.0
SEO,9964.0,22162.0,37383.0,22324.0,42854.0,44944.0,79149.0,29114.0,46823.0,76218.0,52325.0,32947.0
Direct,18566.0,75351.0,51936.0,91368.0,111064.0,117537.0,143589.0,62812.0,138358.0,123378.0,50419.0,63592.0
CPA,21083.0,79331.0,52510.0,113079.0,28620.0,44024.0,49377.0,60115.0,43016.0,83711.0,132252.0,65220.0


In [77]:
dr.head(1)

,_id,type,createdAt,updatedAt,status_x,borrower,partner,product,statusFields,whence_x,adapterName,clickId,id_x,dt,productId_x,product_type,product_name,userId_x,phone,indexName_x,fallbackResponse_x,json_x,id_y,eventType_x,source_x,params_x,timestamp_x,gaId_x,userId_y,utm_campaign_x,utm_medium_x,utm_source_x,utm_term_x,productType_x,yandexClientID_x,productId_y,whence_y,transitionId,partnerName,payout,status_y,transaction_id,externalId,orderId,transactionId,time_x,date,userId,utm_campaign_y,utm_medium_y,utm_source_y,utm_term_y,utm_content,gaId_y,yandexClientID_y,sourceGlobal_x,score,whence_source,registration,Целевое действие,source_type,source_ext,reg_time,indexName_y,fallbackResponse_y,json_y,id,eventType_y,source_y,params_y,timestamp_y,gaId,utm_campaign,utm_medium,utm_source,utm_term,productType_y,yandexClientID,currentPage,sourceGlobal_y,time_y,dt_reg,currentPage1,currentPage2,month
0,5ff2312053668d0001f5b3b9,LINK,2021-01-03 21:03:28.953,2021-01-03 21:03:28.953,NEW,"{'_id': None, 'userId': '5ff231209bad3e0001d75cc9'}",Тинькофф Банк,"{'_id': 5dcd77050d00b700010cfb9c, 'name': 'Tinkoff Drive', 'productType': 'CREDIT_CARD', 'productName': 'Кредитная карта'}",{},vbrru,NaN,NaN,5ff2312053668d0001f5b3b9,2021-01-04,5dcd77050d00b700010cfb9c,CREDIT_CARD,Tinkoff Drive,5ff231209bad3e0001d75cc9,NaN,NaN,NaN,NaN,NaN,NaN,Vbr.ru ссылки,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,vbrru,NaN,Выдача,whence_source,vbrru,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,2021-01


In [78]:
# dr['Доход'] = dr.apply(lambda row: AR_count(row['score'], row['productType_x'], row['status_x']), axis=1)
# # Тут костыли для улучшения прогноза
#dr['Доход'] = dr.apply(lambda row: row['Доход']/3 if row['type']=='API' else row['Доход'],axis=1)
# dr['Доход'] = dr['Доход'].apply(lambda x: x*3)
# dr['Доход'] = dr.apply(lambda row: row['Доход']/8 if row['source_x']=='exRejected' else row['Доход'],axis=1)
# dr['Доход'] = dr.apply(lambda row: row['Доход'] if row['partner'] in ['еКапуста','Деньги Сразу','Веб-Займ']  and row['status_x'] in ['APPROVED', 'ISSUED'] else row['Доход'],axis=1)
# dr['Доход'] = dr.apply(lambda row: row['Доход']/2 if row['source_x'] in ['Bankiros.ru ссылки']  and
#                        row['status_x'] not in ['APPROVED', 'ISSUED'] else row['Доход'],axis=1)
# dr['Доход'] = dr.apply(lambda row: row['Доход']*3 if row['source_x'] in ['Tele2 // Микрозаймы']  and
#                        row['status_x'] not in ['APPROVED', 'ISSUED'] else row['Доход'],axis=1)

In [79]:

forecast = dr[dr['status_x'].isin(['APPROVED','ISSUED'])].groupby(['month','source_x'],as_index=False).agg({"_id": 'count',
                                  "payout": 'sum'})

In [80]:
forecast['payout'] = forecast['payout'].astype('int')
forecast

,month,source_x,_id,payout
0,2021-01,Admitad,25,21460
1,2021-01,Adv.Cake,34,19665
2,2021-01,Bankiros.ru ссылки,323,255421
3,2021-01,Brobank.ru,22,14160
4,2021-01,Cpahub,2,235
5,2021-01,Email,44,46692
6,2021-01,Leadgid,4,570
7,2021-01,Rubid,2,2290
8,2021-01,SMS рекламные,12,15680
9,2021-01,Tele2 // Микрозаймы,3,585


In [81]:
#forecast = forecast/(date.today().day-1)*monthrange(date.today().year, date.today().month)[1]

In [82]:
#forecast = forecast.astype('int').reset_index()
forecast.columns = ['Месяц','Активность','Заявки факт', 'Выручка факт']
forecast['Расход факт'] = 0
forecast['Расход факт']=forecast.apply(lambda row: round(row['Выручка факт']*0.95)*1 if 
                       row['Активность'] in ['Adv.Cake', 'Город Тройка'] else row['Расход факт'],axis=1)
forecast['Расход факт']=forecast.apply(lambda row: round(row['Выручка факт'])*1 if 
                       row['Активность'] in ['Vbr.ru ссылки', 'Bankiros.ru ссылки'] else row['Расход факт'],axis=1)
forecast['Расход факт']=forecast.apply(lambda row: round(6000)*1 if 
                       row['Активность'] in ['Email', 'Браузерные пуши (web)'] else row['Расход факт'],axis=1)
forecast['Расход факт']=forecast.apply(lambda row: round(row['Выручка факт']*0.85)*1 if 
                       row['Активность'] in ['Brobank.ru', 'Cpahub',
                                             'Exbico', 'Leadgid', 'Admitad',
                                             'Rubid', 'Перфоманс через Артикс'] else row['Расход факт'],axis=1)
forecast['Расход факт']=forecast.apply(lambda row: round(row['Выручка факт']*1.1)*1 if 
                       row['Активность'] in ['SMS рекламные'] else row['Расход факт'],axis=1)

forecast['Расход факт']=forecast.apply(lambda row: round(row['Выручка факт']*0.75)*1 if 
                       row['Активность'] in ['Tele2 // Микрозаймы', 'mega'] else row['Расход факт'],axis=1)
forecast['Расход факт']=forecast.apply(lambda row: round(row['Выручка факт']*0.85)*1 if 
                       row['Активность'] in ['Металлинвест \\ потребы'] else row['Расход факт'],axis=1)
forecast['Расход факт'] =forecast['Расход факт'].astype('int')

In [83]:
forecast['Маржа'] = forecast['Выручка факт'] -forecast['Расход факт'] 
forecast['Отдел'] = forecast['Активность']

forecast['Отдел'] = forecast['Отдел'].apply(lambda x: 'Direct' if 
                                   x in ['Email','SMS рекламные','Браузерные пуши (web)'] else x)
forecast['Отдел'] = forecast['Отдел'].apply(lambda x: 'SEO' if 
                                   x in ['Наполнение лендингов по городам',
                                         'Органический трафик','продвижение МП'] else x)
forecast['Отдел'] = forecast['Отдел'].apply(lambda x: 'SEO' if 
                                   x in ['Наполнение лендингов по городам',
                                         'Органический трафик','продвижение МП',
                                         'накрутка поведенческих факторов',
                                         'Мобильное приложение',
                                         'Новая карточка',
                                         'Новые витрины'] else x)
forecast['Отдел'] = forecast['Отдел'].apply(lambda x: 'CPA' if 
                                   x in ['Adv.Cake','Admitad','Leadgid','Cpahub','Rubid','Город Тройка',
                                         'Мобильное приложение',
                                         'Металлинвест \ потребы','CPA пиар','Новые продукты \\ дебетовые',
                                         'CallTat','Металлинвест \\ РКО','Промокоды','Чек Скан'] else x)
forecast['Отдел'] = forecast['Отдел'].apply(lambda x: 'Своя CPA' if 
                                   x in ['Bankiros.ru ссылки','Vbr.ru WL','Vbr.ru ссылки','Brobank.ru',
                                         'Металлинвест','Bankiros.ru WL','Новые партнеры WL'] else x)
forecast['Отдел'] = forecast['Отдел'].apply(lambda x: 'Контент' if 
                                   x in ['Яндекс.Дзен','Статьи на сайте',
                                         'Рекламные статьи (платные публикации)'] else x)
forecast['Отдел'] = forecast['Отдел'].apply(lambda x: 'Крупные партнеры' if 
                                   x in ['Tele2 // Микрозаймы','Теле2 \\ Кредитные карты',
                                         'Qiwi \\ Микрозайм','New (2Gis)','Мегафон','Крупный партнер (1)',
                                         'Крупный партнер (2)','Крупный партнер (3)'] else x)
forecast['Отдел'] = forecast['Отдел'].apply(lambda x: 'КЦ' if 
                                   x in ['Exbico','Внешние базы','Обзвон роботом','Допродажи КЦ'] else x)
forecast['Отдел'] = forecast['Отдел'].apply(lambda x: 'Перфоманс' if 
                                   x in ['Перфоманс','Тендерное агентство (Артикс)','МТС \\  баннер + СМС',
                                         'Пуш партнерка','Тестирование гипотез'] else x)
forecast['Отдел'] = forecast['Отдел'].apply(lambda x: 'Продукт' if 
                                   x in ['Быстрая регистрация','Подписка','Конкурс',
                                         'Энвибокс','Новые продукты \\ Подборщик дебетовый карт',
                                         'Новые продукты \\ РКО'] else x)
forecast['Отдел'] = forecast['Отдел'].apply(lambda x: 'Продукт' if 
                                   x in ['Быстрая регистрация','Подписка','Конкурс',
                                         'Энвибокс','Новые продукты \\ Подборщик дебетовый карт',
                                         'Новые продукты \\ РКО'] else x)
forecast['Отдел'] = forecast['Отдел'].apply(lambda x: 'Мобильное приложение 1' if 
                                   x in ['Мобильное приложение 1'] else x)
forecast = forecast[forecast['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено',
                                'Контент','КЦ'])]
forecast

,Месяц,Активность,Заявки факт,Выручка факт,Расход факт,Маржа,Отдел
0,2021-01,Admitad,25,21460,18241,3219,CPA
1,2021-01,Adv.Cake,34,19665,18682,983,CPA
2,2021-01,Bankiros.ru ссылки,323,255421,255421,0,Своя CPA
3,2021-01,Brobank.ru,22,14160,12036,2124,Своя CPA
4,2021-01,Cpahub,2,235,200,35,CPA
5,2021-01,Email,44,46692,6000,40692,Direct
6,2021-01,Leadgid,4,570,484,86,CPA
7,2021-01,Rubid,2,2290,1946,344,CPA
8,2021-01,SMS рекламные,12,15680,17248,-1568,Direct
9,2021-01,Tele2 // Микрозаймы,3,585,439,146,Крупные партнеры


## Маркетинг

In [84]:
marketing = forecast.pivot_table(index='Отдел',columns='Месяц',values='Расход факт').sort_index(axis='columns',
                                                                                level='Месяц',
                                                                                ascending=False)
marketing=marketing.sort_values('Отдел',ascending=False)


marketing.to_excel('Ежемесячный отчет/marketing_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1AVRkJrGTznlhpezr_2YxxtoyEjunlI_dP97uSCM-wx0')
worksheet = sh.get_worksheet(15) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Ежемесячный отчет/marketing_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)
marketing

Месяц,2021-12,2021-11,2021-10,2021-09,2021-08,2021-07,2021-06,2021-05,2021-04,2021-03,2021-02,2021-01
Отдел,,,,,,,,,,,,
Своя CPA,14070.0,52375.0,71142.5,69523.0,77758.000000,99357.00,66125.0,58849.666667,81896.500000,157240.333333,104076.666667,102569.666667
Перфоманс,NaN,0.0,0.0,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,NaN
Мобильное приложение 1,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
Крупные партнеры,5416.0,94472.0,51412.0,106529.0,53977.500000,83336.00,103751.0,45750.000000,54399.000000,115898.000000,37876.000000,439.000000
Контент,NaN,0.0,0.0,0.0,0.000000,0.00,0.0,NaN,0.000000,0.000000,0.000000,0.000000
SEO,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
Direct,4930.5,18286.5,16318.5,10171.0,24208.000000,28817.00,27724.5,3334.000000,9056.333333,12725.333333,8924.333333,9749.333333
CPA,9701.0,23608.0,16185.0,19796.2,8908.666667,9749.75,8906.0,10477.000000,9362.750000,18416.750000,19175.333333,9567.166667


In [85]:
marzha = forecast.pivot_table(index='Отдел',columns='Месяц',values='Маржа').sort_index(axis='columns',
                                                                                level='Месяц',
                                                                                ascending=False)
marzha=marzha.sort_values('Отдел',ascending=False)

marzha=marzha.fillna(0)
marzha.to_excel('Ежемесячный отчет/marzha_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1AVRkJrGTznlhpezr_2YxxtoyEjunlI_dP97uSCM-wx0')
worksheet = sh.get_worksheet(16) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Ежемесячный отчет/marzha_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)
marzha

Месяц,2021-12,2021-11,2021-10,2021-09,2021-08,2021-07,2021-06,2021-05,2021-04,2021-03,2021-02,2021-01
Отдел,,,,,,,,,,,,
Своя CPA,0.0,292.500000,135.000000,197.0,397.000000,383.00,397.5,395.333333,544.50,700.000000,167.000000,708.000000
Перфоманс,0.0,3995.000000,6246.000000,225.0,1460.000000,1240.00,12455.0,2697.000000,10460.00,2880.000000,430.000000,0.000000
Мобильное приложение 1,36940.0,55374.000000,47930.000000,47250.0,60350.000000,19175.00,19629.0,24873.000000,36540.00,6574.000000,280.000000,280.000000
Крупные партнеры,1805.0,31491.000000,21537.500000,35510.0,19297.500000,30739.00,34584.0,15250.000000,19533.00,38632.000000,12625.000000,146.000000
Контент,0.0,300.000000,900.000000,860.0,500.000000,3660.00,900.0,0.000000,280.00,4222.000000,15260.000000,1760.000000
SEO,9964.0,22162.000000,37383.000000,22324.0,42854.000000,44944.00,79149.0,29114.000000,46823.00,76218.000000,52325.000000,32947.000000
Direct,4352.5,19389.000000,9649.500000,20285.0,31324.000000,29951.50,44070.0,28072.000000,37063.00,28400.666667,7882.000000,11448.000000
CPA,840.5,2835.666667,1318.333333,2819.6,631.333333,1256.25,969.4,1546.000000,1391.25,2511.000000,2866.666667,1302.833333


In [86]:
forecast

,Месяц,Активность,Заявки факт,Выручка факт,Расход факт,Маржа,Отдел
0,2021-01,Admitad,25,21460,18241,3219,CPA
1,2021-01,Adv.Cake,34,19665,18682,983,CPA
2,2021-01,Bankiros.ru ссылки,323,255421,255421,0,Своя CPA
3,2021-01,Brobank.ru,22,14160,12036,2124,Своя CPA
4,2021-01,Cpahub,2,235,200,35,CPA
5,2021-01,Email,44,46692,6000,40692,Direct
6,2021-01,Leadgid,4,570,484,86,CPA
7,2021-01,Rubid,2,2290,1946,344,CPA
8,2021-01,SMS рекламные,12,15680,17248,-1568,Direct
9,2021-01,Tele2 // Микрозаймы,3,585,439,146,Крупные партнеры


In [87]:
forecast['cost_income']= (((forecast['Выручка факт']/forecast['Расход факт'])*100).round(2))
forecast

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Месяц,Активность,Заявки факт,Выручка факт,Расход факт,Маржа,Отдел,cost_income
0,2021-01,Admitad,25,21460,18241,3219,CPA,117.65
1,2021-01,Adv.Cake,34,19665,18682,983,CPA,105.26
2,2021-01,Bankiros.ru ссылки,323,255421,255421,0,Своя CPA,100.00
3,2021-01,Brobank.ru,22,14160,12036,2124,Своя CPA,117.65
4,2021-01,Cpahub,2,235,200,35,CPA,117.50
5,2021-01,Email,44,46692,6000,40692,Direct,778.20
6,2021-01,Leadgid,4,570,484,86,CPA,117.77
7,2021-01,Rubid,2,2290,1946,344,CPA,117.68
8,2021-01,SMS рекламные,12,15680,17248,-1568,Direct,90.91
9,2021-01,Tele2 // Микрозаймы,3,585,439,146,Крупные партнеры,133.26


In [88]:
forecast.replace([np.inf, -np.inf], np.nan, inplace=True)


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [89]:
forecast = forecast.fillna(0)

In [90]:
forecast

,Месяц,Активность,Заявки факт,Выручка факт,Расход факт,Маржа,Отдел,cost_income
0,2021-01,Admitad,25,21460,18241,3219,CPA,117.65
1,2021-01,Adv.Cake,34,19665,18682,983,CPA,105.26
2,2021-01,Bankiros.ru ссылки,323,255421,255421,0,Своя CPA,100.00
3,2021-01,Brobank.ru,22,14160,12036,2124,Своя CPA,117.65
4,2021-01,Cpahub,2,235,200,35,CPA,117.50
5,2021-01,Email,44,46692,6000,40692,Direct,778.20
6,2021-01,Leadgid,4,570,484,86,CPA,117.77
7,2021-01,Rubid,2,2290,1946,344,CPA,117.68
8,2021-01,SMS рекламные,12,15680,17248,-1568,Direct,90.91
9,2021-01,Tele2 // Микрозаймы,3,585,439,146,Крупные партнеры,133.26


In [91]:
cost_income = forecast.pivot_table(index='Отдел',columns='Месяц',values='cost_income').sort_index(axis='columns',
                                                                                level='Месяц',
                                                                                ascending=False)
cost_income=cost_income.sort_values('Отдел',ascending=False)
cost_income=cost_income.fillna(0)
cost_income.to_excel('Ежемесячный отчет/cost_income_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1AVRkJrGTznlhpezr_2YxxtoyEjunlI_dP97uSCM-wx0')
worksheet = sh.get_worksheet(17) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Ежемесячный отчет/cost_income_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)
cost_income

Месяц,2021-12,2021-11,2021-10,2021-09,2021-08,2021-07,2021-06,2021-05,2021-04,2021-03,2021-02,2021-01
Отдел,,,,,,,,,,,,
Своя CPA,100.00,108.825000,108.825000,108.810000,108.830000,108.8150,108.825,105.88,108.825000,105.883333,105.883333,105.883333
Перфоманс,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000,0.00,0.000000,0.000000,0.000000,0.000000
Мобильное приложение 1,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000,0.00,0.000000,0.000000,0.000000,0.000000
Крупные партнеры,133.33,133.330000,66.665000,133.330000,66.665000,66.6650,133.330,133.33,66.665000,133.330000,133.330000,133.260000
Контент,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000,0.00,0.000000,0.000000,0.000000,0.000000
SEO,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000,0.00,0.000000,0.000000,0.000000,0.000000
Direct,170.92,441.765000,276.465000,444.403333,649.655000,633.7650,867.415,563.81,722.346667,583.536667,235.796667,296.480000
CPA,111.45,113.516667,113.486667,115.174000,113.516667,114.5525,115.198,115.17,114.550000,114.552500,115.510000,115.585000


In [92]:
df_traf

,Отдел,month,visits
0,213,2021-11,1.0
1,CPA,2021-01,7785.0
2,CPA,2021-02,6517.0
3,CPA,2021-03,6899.0
4,CPA,2021-04,5971.0
5,CPA,2021-05,7963.0
6,CPA,2021-06,4528.0
7,CPA,2021-07,3178.0
8,CPA,2021-08,2738.0
9,CPA,2021-09,16337.0


In [93]:
dr_zayavki1= dr_zayavki.groupby(['Отдел','month'],as_index=False).agg({'dt':'sum'})


In [94]:
epc = df_traf.merge(dr_zayavki1,how='left',on=['Отдел','month'])
epc['epc'] = ((epc['dt']/epc['visits'])*100).round(2)
epc=epc[epc['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено',
                                'Контент','КЦ'])].pivot_table(index='Отдел',
                                                              columns='month',
                                                              values='epc').sort_index(axis='columns',
                                                                                level='month',
                                                                                ascending=False)
epc=epc.sort_values('Отдел',ascending=False)

epc = epc.fillna(0)
epc.to_excel('Ежемесячный отчет/epc_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1AVRkJrGTznlhpezr_2YxxtoyEjunlI_dP97uSCM-wx0')
worksheet = sh.get_worksheet(18) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Ежемесячный отчет/epc_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)
epc

month,2021-12,2021-11,2021-10,2021-09,2021-08,2021-07,2021-06,2021-05,2021-04,2021-03,2021-02,2021-01
Отдел,,,,,,,,,,,,
Своя CPA,417.65,2090.91,1075.86,1609.52,832.63,820.72,209.95,285.10,401.57,757.09,348.44,1406.74
Перфоманс,0.00,0.56,0.68,0.11,0.47,0.38,0.31,0.61,0.86,2.51,2.20,0.30
Мобильное приложение 1,13.11,12.59,13.61,11.90,10.01,7.26,5.99,6.33,11.09,9.08,4.39,1.47
Крупные партнеры,2.35,2.82,4.49,4.92,6.98,7.95,6.11,5.09,5.19,5.02,3.84,5.19
Контент,175.00,187.50,175.00,171.43,23.81,170.00,145.45,26.19,310.00,286.36,57.89,120.00
КЦ,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.88
SEO,1.18,1.42,1.42,1.53,2.17,4.03,5.09,5.71,7.63,7.25,6.65,3.69
Direct,33.80,17.41,61.08,51.57,103.65,80.97,148.14,34.46,22.34,21.44,40.20,41.96
CPA,7.02,7.03,8.20,2.48,9.20,11.04,12.77,13.31,9.13,16.45,19.33,10.08


In [95]:
dr_zayavki1

,Отдел,month,dt
0,CPA,2021-01,785
1,CPA,2021-02,1260
2,CPA,2021-03,1135
3,CPA,2021-04,545
4,CPA,2021-05,1060
5,CPA,2021-06,578
6,CPA,2021-07,351
7,CPA,2021-08,252
8,CPA,2021-09,405
9,CPA,2021-10,431


In [96]:
d3_vyr = d3[d3['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено',
                                'Контент','КЦ'])].groupby(['Отдел','month'],as_index=False).agg({'payout':'sum'})
d3_vyr                            

,Отдел,month,payout
0,CPA,2021-01,65220
1,CPA,2021-02,132252
2,CPA,2021-03,83711
3,CPA,2021-04,43016
4,CPA,2021-05,60115
5,CPA,2021-06,49377
6,CPA,2021-07,44024
7,CPA,2021-08,28620
8,CPA,2021-09,113079
9,CPA,2021-10,52510


In [97]:
dr_zayavki1 = dr_zayavki1.merge(d3_vyr,how='left',on=['Отдел','month'])
dr_zayavki1

,Отдел,month,dt,payout
0,CPA,2021-01,785,65220.0
1,CPA,2021-02,1260,132252.0
2,CPA,2021-03,1135,83711.0
3,CPA,2021-04,545,43016.0
4,CPA,2021-05,1060,60115.0
5,CPA,2021-06,578,49377.0
6,CPA,2021-07,351,44024.0
7,CPA,2021-08,252,28620.0
8,CPA,2021-09,405,113079.0
9,CPA,2021-10,431,52510.0


In [98]:
dr_zayavki1['epl'] =(dr_zayavki1['payout']/dr_zayavki1['dt']).round(2)
dr_zayavki1

,Отдел,month,dt,payout,epl
0,CPA,2021-01,785,65220.0,83.08
1,CPA,2021-02,1260,132252.0,104.96
2,CPA,2021-03,1135,83711.0,73.75
3,CPA,2021-04,545,43016.0,78.93
4,CPA,2021-05,1060,60115.0,56.71
5,CPA,2021-06,578,49377.0,85.43
6,CPA,2021-07,351,44024.0,125.42
7,CPA,2021-08,252,28620.0,113.57
8,CPA,2021-09,405,113079.0,279.21
9,CPA,2021-10,431,52510.0,121.83


In [99]:
epl = dr_zayavki1[dr_zayavki1['Отдел'].isin(['CPA','Direct','Перфоманс',
                                      'Своя CPA','Мобильное приложение 1','SEO',
                                     'Крупные партнеры','CPA','Не определено',
                                'Контент','КЦ'])].pivot_table(index='Отдел',
                                                              columns='month',
                                                              values='epl').sort_index(axis='columns',
                                                                                level='month',
                                                                                ascending=False)
epl=epl.sort_values('Отдел',ascending=False)
epl.to_excel('Ежемесячный отчет/epl_Alexander.xlsx')
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1AVRkJrGTznlhpezr_2YxxtoyEjunlI_dP97uSCM-wx0')
worksheet = sh.get_worksheet(19) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Ежемесячный отчет/epl_Alexander.xlsx'
                  )
set_with_dataframe(worksheet, df)
epl

month,2021-12,2021-11,2021-10,2021-09,2021-08,2021-07,2021-06,2021-05,2021-04,2021-03,2021-02,2021-01
Отдел,,,,,,,,,,,,
Своя CPA,198.17,152.66,456.91,206.27,197.61,218.97,286.73,206.43,161.02,216.55,200.34,247.47
Перфоманс,NaN,235.00,160.15,32.14,63.48,77.50,265.00,79.32,249.05,125.22,15.93,NaN
Мобильное приложение 1,96.20,99.06,91.30,98.23,156.35,88.36,114.12,116.77,57.00,32.54,8.00,35.00
Крупные партнеры,138.87,295.00,451.70,579.75,384.65,422.50,294.33,374.23,598.64,466.86,701.40,146.25
Контент,NaN,20.00,128.57,71.67,100.00,215.29,56.25,NaN,9.03,67.02,693.64,58.67
SEO,66.87,78.31,131.63,100.11,162.94,142.23,153.99,55.77,63.02,112.09,105.28,100.76
Direct,152.18,214.68,200.53,371.41,279.06,260.61,225.41,204.60,261.05,190.99,111.30,95.92
CPA,97.61,148.84,121.83,279.21,113.57,125.42,85.43,56.71,78.93,73.75,104.96,83.08
